<h1>Part 1 - Intro</h1><br>
Hi there, welcome to NumpyChess, this is my little demo/learning project where I screw around with the numpy, pyplot and tensorflow libraries. I just thought that I'd make something a little more interesting than 'Hello World!'. So the goal here is to write a chess game where you can play against the computer and hopefully lose from first principals. I realize there's lots of libraries out there already to do this, but as a learning exercise, I'll try to do this from scratch. Eventually, once the model and game are up and running, port it to Android so I can play on my phone against anyone locally through Bluetooth, because I think it would be nice to have a chess board that can be played from different tents camping with no reception. And I think it would be amusing to be able to write something that can beat me in chess from first principles (I am by no means an expert player or even a competitive player, just a recreational one).<br>
<br>
If you've never encountered this format before, it's a Jupyter Notebook, which runs Python. It's a sequence of markdown and code cells. The code all shares a common execution environment like you would expect from Python, but you can go and rerun cells in whatever order you like. Click on Edit to open a runnable window and Run All if you want to run everything sequentially.<br>
<br>
So let's break that down a bit better:<br>
<br>
1. Write a basic chess game - DONE<br>
2. Build the datasets/features - IN PROGRESS<br>
3. Build and train the models<br>
4. Plug it all together<br>
5. Play chess<br>
6. Refactor and polish - IN PROGRESS<br>
7. Port to Android<br>
<br>
Current progress:<br>
It could use more tests, but otherwise the game appears to be working. I'll do better test coverage when I rewrite it, because this is a prototype, not something I'm happy with (see Part 6 for more details). The UI sucks in that it simply prints a new board and you have to type your moves in manually, no mouse clicking because Kaggle has issues with interactivity. But, as there's still plenty of other things in here to look at, I'll solve having an interactive board another day, or maybe I won't and just have the log history of the game and put that in the Android version. So for now, it just prints a new board after every move and you have to type your move.<br>
<br>
Next up is to turn the data into something we can do something with, namely some numpy arrays of shape (num_turns, rows, columns) with labels of who won. I'll try to engineer some features from there and see where I get. To get the arrays, I'm going to replay a bunch of other peoples' games and grab the arrays at the end and label them accordingly. Fortunately, chess is one of those games where data is available freely, in abundance, and of pretty high quality. There's absolutely no shortage of online repositories of chess games, and the one I'll be using is Lichess.org's open database. And once we look at the data, we find that there's multiple formats available, any of which can be used, but all require parsers, with some being simpler to write than others. In particular, Universal Chess Interface (UCI) is the one with the easiest parsing in my opinion, so that's the one I'm going to start with.<br>
<br>
Update: I've decided that the original code is so bad that I'm going to rewrite it and include the parser at that point. This is partly due to how slow it would be, since ideally we'd be loading millions of games. This could be done without error checking and much faster, but that just sounds like a good way to get bad data fast, so I'll test my code and convert the games to numpy arrays at the same time.<br>
<br>
<br>
**Note: If you would just like to play the game yourself, first select 'Edit' and then click 'Run All', and then navigate down to Part 3 (the Table of Contents works in Edit mode) and uncomment the line indicated and run that cell again.**

<h1>Part 2 - The Code</h1><br>
My imports and global variables. I'm going to try to write this using numpy, pyplot and tensorflow. I have a preference for using type hints in python, so I'll be doing that where I remember to. I'll come back for a refactor polishing where I'll clean that all up. Other than that, there's just the factored out magic numbers into meaningful names.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import unittest

from enum import Enum
from typing import Optional, Tuple, List, Dict
from numpy import ndarray
from matplotlib.patches import Rectangle
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.widgets import Button

# Define piece type ordinals (0 = empty, 1 = pawn, 2 = rook, 3 = knight, 4 = bishop, 5 = queen, 6 = king)
EMPTY, PAWN, ROOK, KNIGHT, BISHOP, QUEEN, KING = range(7)
# Define color ordinals (0 = white, 1 = black)
WHITE, BLACK = 1, 2
# move-tuple indexes
MOVE_START_ROW, MOVE_START_COL, MOVE_END_ROW, MOVE_END_COL = 0, 1, 2, 3
# piece indexes
TYPE, COLOR = 0, 1
# position indexes
LOCATION_ROW, LOCATION_COLUMN = 0, 1
# move directions for pieces
BISHOP_MOVE_DIRECTIONS = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
ROOK_MOVE_DIRECTIONS = [(1, 0), (-1, 0), (0, 1), (0, -1)]
ROYAL_MOVE_DIRECTIONS = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]

<h2>2.1 - The game board</h2>
I'm going to start with a basic game with a numpy array for a board. This array needs to have at least one dimension for rows, one for columns, one for turns, and one for whatever data we want per square to represent our game. In this case, I'm starting with just piece and color information, both of which will be encoded categorically as integers. This will give us a final numpy array of shape (turns, rows, columns, 2) of dtype=int. This format should be easily ingestible by a model.<br>
I'll still need a few other variables to make life a bit easier on ourselves as we go, tracking things like whose turn it is, turn number and winner. Other than that, I'm including the option to load a specific board or game, whether to unit test a scenario or a loaded save game, but that's for later. For now, I'll just start with a standard new game layout with white at the bottom.

In [ ]:
# let's start with making a class for our game
class NumpyChess:
    def __init__(self, starting_board: ndarray = None, player_turn: str = 'white'):
        # a starting scenario or custom game can be passed in as an array
        if starting_board:
            self.board: ndarray = starting_board
        # but if not, we will intialize a normal new game of chess
        else:
            self.board: ndarray = NumpyChess.new_board()
    
        self.player_color: int = WHITE if player_turn.lower() == 'white' else BLACK
        self.opponent_color: int = BLACK if player_turn.lower() == 'white' else WHITE
        self.turn_number: int = self.board.shape[0] - 1
        self.winner: Optional[int] = None

    # returns a new board in the standard layout, ready to start a new game
    @staticmethod
    def new_board() -> ndarray:
        board: ndarray = np.zeros((1, 8, 8, 2), dtype=int)  # 1 layer, 8x8 board, 2 values per piece [type, color]
    
        # Place pawns
        board[0, 1, :, :] = [PAWN, WHITE]
        board[0, 6, :, :] = [PAWN, BLACK]
    
        # Place rooks
        board[0, 0, [0, 7], :] = [ROOK, WHITE]
        board[0, 7, [0, 7], :] = [ROOK, BLACK]
    
        # Place knights
        board[0, 0, [1, 6], :] = [KNIGHT, WHITE]
        board[0, 7, [1, 6], :] = [KNIGHT, BLACK]
    
        # Place bishops
        board[0, 0, [2, 5], :] = [BISHOP, WHITE]
        board[0, 7, [2, 5], :] = [BISHOP, BLACK]
    
        # Place queens
        board[0, 0, 3, :] = [QUEEN, WHITE]
        board[0, 7, 3, :] = [QUEEN, BLACK]
    
        # Place kings
        board[0, 0, 4, :] = [KING, WHITE]
        board[0, 7, 4, :] = [KING, BLACK]
    
        return board

<h2>2.2 - Class and utility methods</h2>
Some assorted utility methods for converting from a tuple of indices such as (0, 3) to a chess location such as 'A4' and back, and one to get moves interactively from the user.

In [ ]:
# converts an index tuple into a letter and number
# examples: (0, 0) -> A1, (1, 2) -> B3
@staticmethod
def index_to_chess_format(index_tuple: Tuple[int, int]) -> str:
    letter: str = chr(index_tuple[1] + 65)
    number: str = str(index_tuple[0] + 1)
    return letter + number

# converts a letter and number into an index tuple
# examples: A1 -> (0, 0), B3 -> (1, 2)
@staticmethod
def chess_to_index_format(position: str) -> Tuple[int, int]:
    if len(position) != 2:
        print(position, 'is invalid')
        return None
    
    lower_position: str = position.lower()
    column_number: int = ord(lower_position[LOCATION_ROW]) - ord('a')
    row_number: int = int(position[LOCATION_COLUMN]) - 1
    
    if column_number < 0 or column_number > 7 or row_number < 0 or row_number > 7:
        print('Invalid row and column,', position, 'is invalid')
        return None
    
    return (row_number, column_number)

# returns the start and end points as a tuple in the form of (start_row, start_column, end_row, end_column)
# TODO: refactor this to include getting the pawn promotion choice if necessary so unit tests can be written for pawn promotion
@staticmethod
def get_next_move_interactive() -> Tuple[int, int, int, int]:
    while True:
        start: str = input("Enter start position (example, e2): ")
        start_tuple: Tuple[int, int] = chess_to_index_format(start)
        if not start_tuple:
            print('invalid piece location')
            continue

        end: str = input("Enter end position (example, e4): ")
        end_tuple: Tuple[int, int] = chess_to_index_format(end)
        if not end_tuple:
            print('invalid piece location')
            continue

        return start, end
    
NumpyChess.index_to_chess_format = index_to_chess_format
NumpyChess.chess_to_index_format = chess_to_index_format
NumpyChess.get_next_move_interactive = get_next_move_interactive

<h2>2.3 - The game loop</h2>
Now that we can get new turn input from the user and can convert it more naturally to the indices the arrays will use, we can start with our game loop and work backwards from there.

In [ ]:
# if you want to play a game interactively, or start being interactive after a certain point, calling this will start the game and display the board
def start_game_interactive(self) -> None:
    self.display_board()
    
    while not self.winner:
        print('Waiting for', 'White' if self.player_color == WHITE else 'Black', 'to move')
        start, end = NumpyChess.get_next_move_interactive()
        board: ndarray = self.take_turn(start, end, logging=False)
        self.display_board()
        
    print('Game Over.')
            
# to take a turn programmatically, no display, it returns the new board position if it was a valid move, None if it is not
def take_turn(self, start_location: str, end_location: str, logging: bool = False) -> ndarray:
    start_move: Tuple[int, int] = NumpyChess.chess_to_index_format(start_location)
    end_move: Tuple[int, int] = NumpyChess.chess_to_index_format(end_location)
    
    if not start_move or not end_move:
        print('Invalid move, try again')
        return None
    
    move: Tuple[int, int, int, int] = (start_move[0], start_move[1], end_move[0], end_move[1])
    print('Moving from', start_location, 'to', end_location)
    
    next_board: ndarray = self.try_move_piece(move, logging)
    if next_board is not None:
        if logging:
            print('Move successful')
        if is_in_checkmate(self.opponent_color, next_board):
            self.winner = self.player_color
            print('Checkmate!', 'White' if self.winner == WHITE else 'Black', 'wins!')
            return
        if is_in_check(self.opponent_color, next_board):
            print('Check!')
        next_board = next_board.reshape(1, 8, 8, 2)
        self.board = np.concatenate((self.board, next_board), axis=0)
        next_board = next_board.reshape(8, 8, 2)
        self.player_color, self.opponent_color = self.opponent_color, self.player_color
        self.turn_number += 1
        return next_board
    
    print('Invalid move, try again', '\n')
    return None
            
# plug it into our class
NumpyChess.start_game_interactive = start_game_interactive
NumpyChess.take_turn = take_turn

<h2>2.4 - Display the board</h2>
First up in our main game loop is being able to display the board. Using pyplot of course, once again just to screw around. The images we'll be using are the Wikimedia commons chess pieces. I just downloaded the 45x45 pixel ones from Wikipedia, but feel free to find better ones or do a fancy board, etc. I'm not using the reversed ones just yet, but they're there for when I get to it.

In [ ]:
# Load images for chess pieces
piece_image_paths: Dict[Tuple[int, int], str] = {
    (PAWN, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_pawn.png',
    (ROOK, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_rook.png',
    (KNIGHT, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_knight.png',
    (BISHOP, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_bishop.png',
    (QUEEN, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_queen.png',
    (KING, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_king.png',
    (PAWN, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_pawn.png',
    (ROOK, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_rook.png',
    (KNIGHT, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_knight.png',
    (BISHOP, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_bishop.png',
    (QUEEN, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_queen.png',
    (KING, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_king.png',
}

# TODO: make the opponent's pieces reversed
# TODO: allow for toggling which player is on bottom
# TODO: make mode for automatic switch after each turn
reversed_piece_image_paths: Dict[Tuple[int, int], str] = {
    (PAWN, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_pawn_reversed.png',
    (ROOK, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_rook_reversed.png',
    (KNIGHT, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_knight_reversed.png',
    (BISHOP, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_bishop_reversed.png',
    (QUEEN, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_queen_reversed.png',
    (KING, WHITE): '/kaggle/input/wikimedia-commons-chesspieces/white_king_reversed.png',
    (PAWN, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_pawn_reversed.png',
    (ROOK, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_rook_reversed.png',
    (KNIGHT, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_knight_reversed.png',
    (BISHOP, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_bishop_reversed.png',
    (QUEEN, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_queen_reversed.png',
    (KING, BLACK): '/kaggle/input/wikimedia-commons-chesspieces/black_king_reversed.png',
}

piece_images = {key: plt.imread(path) for key, path in piece_image_paths.items()}
reversed_piece_images = {key: plt.imread(path) for key, path in reversed_piece_image_paths.items()}

# Function to display the chessboard graphically with images
def display_board(self, turn=-1):
    fig, ax = plt.subplots(figsize=(6, 6))
    board = self.board[turn, :, :, :]

    # Draw the chessboard
    for row in range(8):
        for col in range(8):
            color = 'white' if (row + col) % 2 == 1 else 'gray'
            ax.add_patch(Rectangle((col, 7-row), 1, 1, color=color))

            piece = board[row, col, :]
            if piece[TYPE] != EMPTY:
                image = piece_images[(piece[TYPE], piece[COLOR])]
                imagebox = OffsetImage(image, zoom=1.0)  # Adjust zoom to fit the image within the square
                ab = AnnotationBbox(imagebox, (col + 0.5, 7-row + 0.5), frameon=False)
                ax.add_artist(ab)

    # Set axis limits and labels
    ax.set_xlim(0, 8)
    ax.set_ylim(0, 8)
    ax.set_xticks(np.arange(8) + 0.5)
    ax.set_yticks(np.arange(8) + 0.5)
    ax.set_xticklabels(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    ax.set_yticklabels(['8', '7', '6', '5', '4', '3', '2', '1'])
    ax.invert_yaxis()
    ax.set_aspect('equal')
    plt.show()

# add our method to the class
NumpyChess.display_board = display_board

<h2>2.5 - Making moves</h2>
Next up in our game loop is to try to make a move. This will entail checking to see that it's a legal move
Next up in our main loop is to try the move. If this is a legal move, we should receive a new board with the move made. If it is not, we should receive None and we would ask for a valid move. This is the core game logic.

In [ ]:
# returns a new ndarray of shape (8, 8, 2) if this is a valid move, None if it is not
def try_move_piece(self, move: Tuple[int, int, int, int], logging: bool = False, promotion_choice: str = 'q') -> Optional[ndarray]:
    start_label = NumpyChess.index_to_chess_format((move[0], move[1]))
    end_label = NumpyChess.index_to_chess_format((move[2], move[3]))
    
    if logging:
        print('Attempting move:', start_label, 'to', end_label)
        
    current_board: ndarray = self.board[-1, :, :, :]
    last_board: ndarray = self.board[-2, :, :, :] if self.board.shape[0] > 1 else None

    # if no starting piece, move is invalid
    starting_piece: ndarray = current_board[move[MOVE_START_ROW], move[MOVE_START_COL]]
    if starting_piece[TYPE] == EMPTY:
        print('No piece at', start_label, 'found:', starting_piece)
        if logging:
            print('current_board:', current_board)
        return None
    
    # make sure the piece is the right color
    if starting_piece[COLOR] != self.player_color:
        print('That is not your piece! Stop trying to cheat you cheater!')
        return None

    # get a list of all possible non-castling moves for this piece (castling is done separately to make check easier to determine)
    # this is the method that determines what is a legal move
    possible_moves: List[Tuple[int, int]] = get_non_castling_moves(current_board, last_board, move[MOVE_START_ROW], move[MOVE_START_COL], logging)
    legal_moves: List[Tuple[int, int]] = []
    row = move[MOVE_START_ROW]
    column = move[MOVE_START_COL]

    if starting_piece[TYPE] == KING:
        if logging:
            print('Checking if the king can castle')
            
        king_start = (0 if starting_piece[COLOR] == WHITE else 7, 4)
        # if king is in check, he cannot castle
        if not is_in_check(self.player_color, current_board):
            # see if the king has ever moved, if so, the rest is unnecessary
            king_never_moved = king_start[0] == row and king_start[1] == column and np.all(self.board[:, king_start[0], king_start[1], TYPE] == KING)
            if logging:
                print('King has never moved:', king_never_moved)
            if king_never_moved:
                # similar check for rooks
                queens_rook_never_moved = np.all(self.board[:, king_start[0], 0] == [ROOK, self.player_color])
                if logging:
                    print('Queens Rook has never moved:', king_never_moved)
                kings_rook_never_moved = np.all(self.board[:, king_start[0], 7] == [ROOK, self.player_color])
                if logging:
                    print('Kings Rook has never moved:', king_never_moved)
                if queens_rook_never_moved:
                    if current_board[row, column - 1, TYPE] == EMPTY and current_board[row, column - 2, TYPE] == EMPTY:
                        between_board = apply_move(current_board, (row, column, row, column - 1))
                        after_board = apply_move(current_board, (row, column, row, column - 2))
                        if not is_in_check(self.player_color, between_board) and not is_in_check(self.player_color, after_board):
                            # we've tested for check, so we can pass it directly to the legal_moves list
                            if logging:
                                print('Queen side castling possible')
                            legal_moves.append((row, column - 2))
                        else:
                            if logging:
                                print('Queen side castling not possible')
                if kings_rook_never_moved:
                    if current_board[row, column + 1, TYPE] == EMPTY and current_board[row, column + 2, TYPE] == EMPTY:
                        between_board = apply_move(current_board, (row, column, row, column + 1))
                        after_board = apply_move(current_board, (row, column, row, column + 2))
                        if not is_in_check(self.player_color, between_board) and not is_in_check(self.player_color, after_board):
                            # we've tested for check, so we can pass it directly to the legal_moves list
                            if logging:
                                print('King side castling possible')
                            legal_moves.append((row, column + 2))
                        else:
                            if logging:
                                print('King side castling not possible')
        else:
            if logging:
                print('Nope, the king is in check')
                            
    # If the destination would not result in the player being in check, then it is a legal move and is added to the legal_moves
    for destination in possible_moves:
        new_board: ndarray = apply_move(current_board, (row, column, destination[LOCATION_ROW], destination[LOCATION_COLUMN]))
        destination_label: str = NumpyChess.index_to_chess_format((destination[LOCATION_ROW], destination[LOCATION_COLUMN]))
        
        if not is_in_check(self.player_color, new_board):
            if logging:
                print(destination_label, 'added to move list')
            legal_moves.append(destination)
        else:
            if logging:
                print(destination_label, 'is not a valid move, the king would be in check')
    
    # if this is not a valid move, fail
    if (move[MOVE_END_ROW], move[MOVE_END_COL]) not in legal_moves:
        bad_move_label = NumpyChess.index_to_chess_format((move[MOVE_END_ROW], move[MOVE_END_COL]))
        print(bad_move_label, 'not found in list of possible moves')
        return None

    # TODO: we need to pass in the promotion choice here if we want to unit test this
    
    new_board: ndarray = apply_move(current_board, move, logging, promotion_choice)
    return new_board

NumpyChess.try_move_piece = try_move_piece

To do this, we're going to get a list of all possible non-castling moves for this piece, not counting whether either player would then be in check. This is the non-castling moves only because testing for castling requires knowing if a king would be in check across a range of locations and must be done afterwards separately.

In [ ]:
def get_non_castling_moves(current_board: ndarray, last_board: ndarray, row: int, column: int, logging: bool = False) -> List[Tuple[int, int]]:
    starting_piece: ndarray = current_board[row, column]
    start_label: str = NumpyChess.index_to_chess_format((row, column))
    player_color: int = starting_piece[COLOR]
    opponent_color: int = WHITE if player_color == BLACK else BLACK
    results: List[Tuple[int, int]] = []
    potential_destinations: List[Tuple[int, int]] = []
        
    if logging:
        print('Getting non-castling moves for', start_label)

    if starting_piece[TYPE] == PAWN:
        move_direction: int = 1 if player_color == WHITE else -1
        last_row: bool = (row == 7) if player_color == WHITE else (row == 0)

        if not last_row:
            # standard 1 move ahead if not blocked
            if current_board[row + move_direction, column][TYPE] == EMPTY:
                if logging:
                    print('1 move ahead added')
                potential_destinations.append((row + move_direction, column))

            # take diagonally forward one square left
            if column > 0:
                target_square: ndarray = current_board[row + move_direction, column - 1]
                if target_square[TYPE] != EMPTY and target_square[COLOR] == opponent_color:
                    if logging:
                        print('Diagonal take left added')
                    potential_destinations.append((row + move_direction, column - 1))

            # take diagonally forward one square right
            if column < 7:
                target_square: ndarray = current_board[row + move_direction, column + 1]
                if target_square[TYPE] != EMPTY and target_square[COLOR] == opponent_color:
                    if logging:
                        print('Diagonal take right added')
                    potential_destinations.append((row + move_direction, column + 1))

        # move forward 2 spaces if on the starting line and it's clear
        original_row: int = 1 if player_color == WHITE else 6
        if row == original_row:
            if current_board[row + move_direction, column][TYPE] == EMPTY:
                if current_board[row + move_direction + move_direction, column][TYPE] == EMPTY:
                    if logging:
                        print('2 spaces forward added')
                    potential_destinations.append((row + move_direction + move_direction, column))

        # En Passant
        # check if the last move was a pawn moving 2 moves forward to land beside our pawn, if so add the en passant move
        # if the last_board is None, we don't care about en passant, we're testing if we're in check
        if last_board is not None and ((row == 4 and player_color == WHITE) or (row == 3 and player_color == BLACK)):
            if logging:
                print('checking for en passant')
            if column > 0:
                # check if we have a pawn to our left of the opposing color now
                pawn_left_now = current_board[row, column - 1]
                check1 = pawn_left_now[TYPE] == PAWN and pawn_left_now[COLOR] == opponent_color
                if logging:
                    print('pawn_left_now:', pawn_left_now, 'check1:', check1)
                
                # check to make sure that the pawn wasn't there last turn
                pawn_left_last = last_board[row, column - 1]
                check2 = pawn_left_last[TYPE] == EMPTY
                if logging:
                    print('pawn_left_last:', pawn_left_last, 'check2:', check2)
                
                # check if our opponent's pawn was in its original spot last turn
                pawn_left_original_last = last_board[1 if opponent_color == WHITE else 6, column - 1]
                check3 = pawn_left_original_last[TYPE] == PAWN and pawn_left_original_last[COLOR] == opponent_color
                if logging:
                    print('pawn_left_original_last:', pawn_left_original_last, 'check3:', check3)
                
                # check if our opponent's pawn's original location is empty this turn
                pawn_left_original_now = current_board[1 if opponent_color == WHITE else 6, column - 1]
                check4 = pawn_left_original_now[TYPE] == EMPTY
                if logging:
                    print('pawn_left_original_now:', pawn_left_original_now, 'check4:', check4)
                
                if check1 and check2 and check3 and check4:
                    dest = (row + move_direction, column - 1)
                    if logging:
                        print('En passant left:', NumpyChess.index_to_chess_format(dest), 'added')
                    potential_destinations.append(dest)
            if column < 7:
                # check if we have a pawn to our right of the opposing color now
                pawn_right_now = current_board[row, column + 1]
                check1 = pawn_right_now[TYPE] == PAWN and pawn_right_now[COLOR] == opponent_color
                if logging:
                    print('pawn_right_now:', pawn_right_now, 'check1:', check1)
                
                # check to make sure that the pawn wasn't there last turn
                pawn_right_last = last_board[row, column + 1]
                check2 = pawn_right_last[TYPE] == EMPTY
                if logging:
                    print('pawn_right_last:', pawn_right_last, 'check2:', check2)
                
                # check if our opponent's pawn was in its original spot last turn
                pawn_right_original_last = last_board[1 if opponent_color == WHITE else 6, column + 1]
                check3 = pawn_right_original_last[TYPE] == PAWN and pawn_right_original_last[COLOR] == opponent_color
                if logging:
                    print('pawn_right_original_last:', pawn_right_original_last, 'check3:', check3)
                
                # check if our opponent's pawn's original location is empty this turn
                pawn_right_original_now = current_board[1 if opponent_color == WHITE else 6, column + 1]
                check4 = pawn_right_original_now[TYPE] == EMPTY
                if logging:
                    print('pawn_right_original_now:', pawn_right_original_now, 'check4:', check4)
                
                if check1 and check2 and check3 and check4:
                    dest = (row + move_direction, column + 1)
                    if logging:
                        print('En passant right:', NumpyChess.index_to_chess_format(dest), 'added')
                    potential_destinations.append(dest)
                    
    elif starting_piece[TYPE] == ROOK:
        for direction in ROOK_MOVE_DIRECTIONS:
            moves_in_direction = add_moves_in_direction(current_board, (row, column), direction, logging=logging)
            potential_destinations.extend(moves_in_direction)

    # TODO: refactor this mess
    elif starting_piece[TYPE] == KNIGHT:
        if row > 0:
            if column > 1:
                target_square = current_board[row - 1, column - 2]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row - 1, column - 2)), 'added')
                    potential_destinations.append((row - 1, column - 2))
            if column < 6:
                target_square = current_board[row - 1, column + 2]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row - 1, column + 2)), 'added')
                    potential_destinations.append((row - 1, column + 2))
        if row > 1:
            if column > 0:
                target_square = current_board[row - 2, column - 1]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row - 2, column - 1)), 'added')
                    potential_destinations.append((row - 2, column - 1))
            if column < 7:
                target_square = current_board[row - 2, column + 1]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row - 2, column + 1)), 'added')
                    potential_destinations.append((row - 2, column + 1))
        if row < 7:
            if column > 1:
                target_square = current_board[row + 1, column - 2]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row + 1, column - 2)), 'added')
                    potential_destinations.append((row + 1, column - 2))
            if column < 6:
                target_square = current_board[row + 1, column + 2]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row + 1, column + 2)), 'added')
                    potential_destinations.append((row + 1, column + 2))
        if row < 6:
            if column > 0:
                target_square = current_board[row + 2, column - 1]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row + 2, column - 1)), 'added')
                    potential_destinations.append((row + 2, column - 1))
            if column < 7:
                target_square = current_board[row + 2, column + 1]
                if target_square[TYPE] == EMPTY or target_square[COLOR] == opponent_color:
                    if logging:
                        print(NumpyChess.index_to_chess_format((row + 2, column + 1)), 'added')
                    potential_destinations.append((row + 2, column + 1))

    elif starting_piece[TYPE] == BISHOP:
        for direction in BISHOP_MOVE_DIRECTIONS:
            moves_in_direction = add_moves_in_direction(current_board, (row, column), direction, logging=logging)
            potential_destinations.extend(moves_in_direction)

    elif starting_piece[TYPE] == QUEEN:
        for direction in ROYAL_MOVE_DIRECTIONS:
            moves_in_direction = add_moves_in_direction(current_board, (row, column), direction, logging=logging)
            potential_destinations.extend(moves_in_direction)

    elif starting_piece[TYPE] == KING:
        for direction in ROYAL_MOVE_DIRECTIONS:
            moves_in_direction = add_moves_in_direction(current_board, (row, column), direction, move_until_end = False, logging=logging)
            potential_destinations.extend(moves_in_direction)
                
    return potential_destinations

def add_moves_in_direction(board: ndarray, start_location: Tuple[int, int], direction: Tuple[int, int], move_until_end = True, logging: bool = False) -> List[Tuple[int, int]]:
    results: List[Tuple[int, int]] = []
    row, col = start_location[0], start_location[1]
    start_piece: ndarray = board[row, col]
    while 0 <= row < 8 and 0 <= col < 8:
        row += direction[0]
        col += direction[1]
        if not (0 <= row < 8 and 0 <= col < 8):
            break
        next_square = board[row, col, :]
        # not empty
        if next_square[TYPE] != EMPTY:
            # if it's an opposing piece it's valid
            if next_square[COLOR] != start_piece[COLOR]:
                if logging:
                    print(NumpyChess.index_to_chess_format((row, column)), 'added, capture')
                results.append((row, col))
            # whether we could capture that piece or not, we can go no further in this direction
            break
        # empty square, so a possible move
        if logging:
            print(NumpyChess.index_to_chess_format((row, column)), 'added, empty')
        results.append((row, col))
        # our king moves only one space, everyone else that uses this goes on until the end
        if not move_until_end:
            break
    return results

Before we can test if a king is in check or not, we need to be able to make a move and analyze the results without impacting the existing game. So apply_move() will apply the move regardless of whether it is a legal move since we'll be using it to test if a king was in check at the end for example. We will take in a board and a move, returning a new board.

In [ ]:
# returns a new board with the move applied, does not validate for legality, we're leaving that to get_possible_moves()
def apply_move(current_board: ndarray, move: Tuple[int, int, int, int], promotion_choice: str = 'q', logging: bool = False) -> ndarray:
    new_board = current_board.copy()
    start_row, start_col, end_row, end_col = move[MOVE_START_ROW], move[MOVE_START_COL], move[MOVE_END_ROW], move[MOVE_END_COL]
    start_piece = current_board[start_row, start_col, :]
    end_piece = current_board[end_row, end_col, :]
    
    # this should never be hit, this should be checked elsewhere, but including as a sanity check
    if start_piece[TYPE] == EMPTY:
        print('No piece to move')
        return None

    # check if this was a castle, move rook accordingly
    if start_piece[TYPE] == KING:
        # queen side (long) castle
        if start_col - end_col == 2:
            new_board[start_row, 3, :] = new_board[start_row, 0, :]
            new_board[start_row, 0, TYPE] = EMPTY
        # king side (short) castle
        elif start_col - end_col == -2:
            new_board[start_row, 5, :] = new_board[start_row, 7, :]
            new_board[start_row, 7, TYPE] = EMPTY

    # check for en passant, remove the taken piece if it is
    if start_piece[TYPE] == PAWN:
        if start_col != end_col and end_piece[TYPE] == EMPTY:
            new_board[start_row, end_col, TYPE] = EMPTY

    # move the piece
    new_board[end_row, end_col, :] = current_board[start_row, start_col, :]
    new_board[start_row, start_col, TYPE] = EMPTY
    
    # check for promotion
    if start_piece[TYPE] == PAWN:
        last_row = 7 if start_piece[COLOR] == WHITE else 0
        if end_row == last_row:
            next_piece: str = input('Promote pawn to which piece? (Q)ueen, (B)ishop, (R)ook, or (K)night')
            next_piece = next_piece.lower()
            if next_piece == 'b':
                new_board[end_row, end_col, TYPE] == BISHOP
            elif next_piece == 'r':
                new_board[end_row, end_col, TYPE] == ROOK
            elif next_piece == 'k':
                new_board[end_row, end_col, TYPE] == KNIGHT
            # NOTE: this is just defaulting to queen, more robust handling could be added later if desired
            else:
                new_board[end_row, end_col, TYPE] == QUEEN

    return new_board

And next we need to see if this is going to put us in check. Because we can't have that. For now, we'll leave that blank while we get the basic movement working, but we'll come back to it.

In [ ]:
def get_king_position(board: ndarray, color: int) -> Tuple[int, int]:
    for row in range(8):
        for col in range(8):
            piece = board[row, col, :]
            if piece[TYPE] == KING and piece[COLOR] == color:
                return (row, col)
            
    # this should be dead code and never happen, the kings should always be on the board
    return None

def is_in_check(color: int, board: ndarray, logging: bool = False) -> bool:
    # our king's position
    king_position = get_king_position(board, color)
    if logging:
        print('king position:', NumpyChess.index_to_chess_format(king_position))
    
    # check if any of our opponents pieces can reach it
    for row in range(8):
        for col in range(8):
            piece = board[row, col, :]
            if piece[COLOR] != color:
                possible_moves = get_non_castling_moves(board, None, row, col)
                # if so, we're in check, return True
                if king_position in possible_moves:
                    return True
                
    # if nobody can reach our king, we are safe and return False
    return False

# TODO
def get_all_piece_locations(color: int, board: ndarray, logging: bool = False):
    results: List[Tuple[int, int]] = []
    for row in range(8):
        for column in range(8):
            if board[row, column, TYPE] != EMPTY and board[row, column, COLOR] == color:
                if logging:
                    label = NumpyChess.index_to_chess_format((row, column))
                    print(label, 'added to piece locations')
                results.append((row, column))
    return results

def is_in_checkmate(color: int, board: ndarray, logging: bool = False) -> bool:
    # if the king isn't in check, we're not in checkmate
    if not is_in_check(color, board):
        if logging:
            print('not in check, so not in checkmate')
        return False
    
    # get all pieces for the player, see if any of their moves results in not being in check
    piece_locations: List[Tuple[int, int]] = get_all_piece_locations(color, board, logging=logging)
    for piece_location in piece_locations:
        all_moves: List[Tuple[int, int]] = get_non_castling_moves(board, None, piece_location[0], piece_location[1], logging=logging)
        for possible_move in all_moves:
            new_board = apply_move(board, (piece_location[0], piece_location[1], possible_move[0], possible_move[1]), logging=logging)
            # there's at least one valid move left, we are not in checkmate
            if not is_in_check(color, new_board):
                return False
            
    # no piece had a possible move that did not leave the king in check, we are in checkmate
    return True

<h1>Part 3 - The game</h1>
Now let's create a game and test it. To play a game, uncomment the code. This is left commented out because Kaggle wants to run the notebook each save and it's interactive, so it would be stupid to run.

In [ ]:
chess_game = NumpyChess()
chess_game.display_board()
# chess_game.start_game_interactive()   # uncomment to play a game, currently no AI, you have to play both sides

<h1>Part 4 - Unit tests</h1><br>

Okay, enough manual testing, let's make some unit tests. I'm only going to test take_turn() for now. If I keep this code, I'll test the inner methods later, but just giving take_turn() a thorough test should get me up and running and it lets me rip apart everything beneath that until I'm happy with it and still keep all of my tests.

<h2>4.1 - Class method tests</h2>
<hr>
<br>
So this is just testing our class methods, namely the constructor and the index to chess format methods.

In [ ]:
class TestClassMethods(unittest.TestCase):
    def test_new_board(self):
        """Test that a new board is initialized correctly."""
        
        chess_game = NumpyChess()
        board = chess_game.new_board()

        # Check that the board is the correct shape
        self.assertEqual(board.shape, (1, 8, 8, 2))

        # Check that pawns are in the correct positions and the correct colors
        self.assertTrue(np.array_equal(board[0, 1, :, 0], [PAWN, PAWN, PAWN, PAWN, PAWN, PAWN, PAWN, PAWN]))  # White pawns are pawns
        self.assertTrue(np.array_equal(board[0, 1, :, 1], [WHITE, WHITE, WHITE, WHITE, WHITE, WHITE, WHITE, WHITE]))  # White pawns are white
        self.assertTrue(np.array_equal(board[0, 6, :, 0], [PAWN, PAWN, PAWN, PAWN, PAWN, PAWN, PAWN, PAWN]))  # Black pawns are pawns
        self.assertTrue(np.array_equal(board[0, 6, :, 1], [BLACK, BLACK, BLACK, BLACK, BLACK, BLACK, BLACK, BLACK]))  # Black pawns are black

        # Check that the back row for white is correct
        self.assertEqual(board[0, 0, 0, 0], ROOK)  # White rook
        self.assertEqual(board[0, 0, 1, 0], KNIGHT)  # White knight
        self.assertEqual(board[0, 0, 2, 0], BISHOP)  # White bishop
        self.assertEqual(board[0, 0, 3, 0], QUEEN)  # White queen
        self.assertEqual(board[0, 0, 4, 0], KING)  # White king
        self.assertEqual(board[0, 0, 5, 0], BISHOP)  # White bishop
        self.assertEqual(board[0, 0, 6, 0], KNIGHT)  # White knight
        self.assertEqual(board[0, 0, 7, 0], ROOK)  # White rook

        # Check that the back row for black is correct
        self.assertEqual(board[0, 7, 0, 0], ROOK)  # Black rook
        self.assertEqual(board[0, 7, 1, 0], KNIGHT)  # Black knight
        self.assertEqual(board[0, 7, 2, 0], BISHOP)  # Black bishop
        self.assertEqual(board[0, 7, 3, 0], QUEEN)  # Black queen
        self.assertEqual(board[0, 7, 4, 0], KING)  # Black king
        self.assertEqual(board[0, 7, 5, 0], BISHOP)  # Black bishop
        self.assertEqual(board[0, 7, 6, 0], KNIGHT)  # Black knight
        self.assertEqual(board[0, 7, 7, 0], ROOK)  # Black rook

    def test_chess_to_index_format(self):
        """Test the conversion from chess notation to index format."""
        
        self.assertEqual(NumpyChess.chess_to_index_format('A1'), (0, 0))
        self.assertEqual(NumpyChess.chess_to_index_format('H8'), (7, 7))
        self.assertEqual(NumpyChess.chess_to_index_format('E2'), (1, 4))
        self.assertEqual(NumpyChess.chess_to_index_format('D4'), (3, 3))

    def test_index_to_chess_format(self):
        """Test the conversion from index format to chess notation."""
        
        self.assertEqual(NumpyChess.index_to_chess_format((0, 0)), 'A1')
        self.assertEqual(NumpyChess.index_to_chess_format((7, 7)), 'H8')
        self.assertEqual(NumpyChess.index_to_chess_format((1, 4)), 'E2')
        self.assertEqual(NumpyChess.index_to_chess_format((3, 3)), 'D4')

# Load and run the test cases
class_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestClassMethods)
unittest.TextTestRunner(verbosity=1).run(class_test_suite)

<h2>4.2 - Pawn tests</h2>
<hr>
<br>
Next up is pawns. This will probably be our biggest section, because we will need duplicated tests for both black and white as pawns are directional. I'm going to go with one, two and three spaces forward, diagonal movement, including capturing the opposing color, without capturing and capturing their own color. I'm also going to add backwards movement attempts, being blocked by either color piece moving forward, en passant, and of course promotion.

In [ ]:
class TestPawns(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()
        
    def test_try_move_piece_white_pawn_move_one_ahead(self):
        """Test a white pawn moving ahead 1."""

        new_board = self.chess_game.take_turn('E2', 'E3')  # White pawn forward one space E2 to E3
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 4, TYPE], PAWN)
        self.assertEqual(new_board[2, 4, COLOR], WHITE)
        self.assertEqual(new_board[1, 4, TYPE], EMPTY)
        
    def test_try_move_piece_white_pawn_move_two_ahead(self):
        """Test a white pawn moving ahead 2."""

        new_board = self.chess_game.take_turn('E2', 'E4')  # White pawn forward two spaces E2 to E4
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[3, 4, TYPE], PAWN)
        self.assertEqual(new_board[3, 4, COLOR], WHITE)
        self.assertEqual(new_board[1, 4, TYPE], EMPTY)
    
    def test_try_move_piece_white_pawn_move_three_ahead(self):
        """Test a white pawn moving ahead 3."""
        
        new_board = self.chess_game.take_turn('E2', 'E5')  # White pawn forward three spaces E2 to E5, invalid move
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_try_move_piece_black_pawn_move_one_ahead(self):
        """Test a black pawn moving ahead 1."""

        self.assertIsNotNone(self.chess_game.take_turn('E2', 'E3'))  # white's first move: e2 to e3
        new_board = self.chess_game.take_turn('E7', 'E6')  # Black pawn forward one space E7 to E6
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[5, 4, TYPE], PAWN)
        self.assertEqual(new_board[5, 4, COLOR], BLACK)
        self.assertEqual(new_board[6, 4, TYPE], EMPTY)
        
    def test_try_move_piece_black_pawn_move_two_ahead(self):
        """Test a black pawn moving ahead 2."""

        self.chess_game.take_turn('E2', 'E3')  # white's first move: e2 to e3
        new_board = self.chess_game.take_turn('E7', 'E5')  # Black pawn forward two spaces E7 to E5
        
        # our new board should show our piece in the new location with the old location empty
        self.assertTrue(new_board[4, 4, TYPE] == PAWN)
        self.assertTrue(new_board[4, 4, COLOR] == BLACK)
        self.assertTrue(new_board[6, 4, TYPE] == EMPTY)
    
    def test_try_move_piece_black_pawn_move_three_ahead(self):
        """Test a black pawn moving ahead 3."""
        
        new_board = self.chess_game.take_turn('E2', 'E3')  # white's first move: e2 to e3
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[2, 4, TYPE] == PAWN)
        self.assertTrue(new_board[2, 4, COLOR] == WHITE)
        self.assertTrue(new_board[1, 4, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E7', 'E4')  # Black pawn forward three spaces E7 to E4, invalid move
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_try_move_piece_white_pawn_backwards(self):
        """Test a white pawn moving backwards."""

        self.chess_game.take_turn('E2', 'E3')  # white pawn: e2 to e3
        new_board = self.chess_game.take_turn('E7', 'E5')  # black pawn: e7 to e5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 4, TYPE] == PAWN)
        self.assertTrue(new_board[4, 4, COLOR] == BLACK)
        self.assertTrue(new_board[6, 4, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E3', 'E2')  # white pawn backwards 1
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_try_move_piece_black_pawn_backwards(self):
        """Test a black pawn moving backwards."""
        
        self.chess_game.take_turn('E2', 'E3')  # white's first move: e2 to e3
        self.chess_game.take_turn('E7', 'E5')  # black's first move: e7 to e5
        new_board = self.chess_game.take_turn('F2', 'F3')  # white pawn forward 1: f2 to f3
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[2, 5, TYPE] == PAWN)
        self.assertTrue(new_board[2, 5, COLOR] == WHITE)
        self.assertTrue(new_board[1, 5, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E5', 'E6') # pawn back 1: e5 to e6
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_en_passant_white(self):
        """Test en passant white."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        self.chess_game.take_turn('D4', 'D5')  # white pawn forward 1: D4 to D5
        self.chess_game.take_turn('C7', 'C5')  # black pawn forward 2: C7 to C5 (vulnerable to en passant)
        new_board = self.chess_game.take_turn('D5', 'C6') # white pawn captures pawn via en passant: D5 to C6
        
        # our new board should show our piece in the new location with the old location empty and the black pawn gone
        self.assertEqual(new_board[5, 2, TYPE], PAWN)
        self.assertEqual(new_board[5, 2, COLOR], WHITE)
        self.assertEqual(new_board[4, 3, TYPE], EMPTY)
        self.assertEqual(new_board[4, 2, TYPE], EMPTY) # black pawn should be gone
        
    def test_en_passant_black(self):
        """Test en passant black."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        self.chess_game.take_turn('D4', 'D5')  # white pawn forward 1: D4 to D5
        self.chess_game.take_turn('E5', 'E4')  # black pawn forward 1: E5 to E4
        self.chess_game.take_turn('F2', 'F4')  # white pawn forward 2: F2 to F4 (vulnerable to en passant)
        new_board = self.chess_game.take_turn('E4', 'F3') # black pawn captures pawn via en passant: E4 to F3
        
        # our new board should show our piece in the new location with the old location empty and the white pawn gone
        self.assertEqual(new_board[2, 5, TYPE], PAWN)
        self.assertEqual(new_board[2, 5, COLOR], BLACK)
        self.assertEqual(new_board[3, 4, TYPE], EMPTY)
        self.assertEqual(new_board[3, 5, TYPE], EMPTY) # white pawn should be gone
        
    def test_capture_white(self):
        """Test normal capture white."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        new_board = self.chess_game.take_turn('D4', 'E5')  # white pawn captures black pawn: D4 to E5
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[4, 4, TYPE], PAWN)
        self.assertEqual(new_board[4, 4, COLOR], WHITE)
        self.assertEqual(new_board[3, 3, TYPE], EMPTY)
        
    def test_capture_black(self):
        """Test normal capture black."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        self.chess_game.take_turn('E2', 'E4')  # white pawn forward 2: E2 to E4
        new_board = self.chess_game.take_turn('E5', 'D4')  # black pawn captures white pawn: E5 to D4
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[3, 3, TYPE], PAWN)
        self.assertEqual(new_board[3, 3, COLOR], BLACK)
        self.assertEqual(new_board[4, 4, TYPE], EMPTY)
        
    def test_white_tries_to_capture_self(self):
        """Test normal capture white."""
        
        self.chess_game.take_turn('D2', 'D3')  # white pawn forward 1: D2 to D3
        new_board = self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 4, TYPE] == PAWN)
        self.assertTrue(new_board[4, 4, COLOR] == BLACK)
        self.assertTrue(new_board[6, 4, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('D4', 'E5')  # white pawn captures black pawn: D4 to E5
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_black_tries_to_capture_self(self):
        """Test normal capture white."""
        
        self.chess_game.take_turn('D2', 'D3')  # white pawn forward 1: D2 to D3
        self.chess_game.take_turn('E7', 'E6')  # black pawn forward 2: E7 to E6
        new_board = self.chess_game.take_turn('D3', 'D4')  # white pawn forward 1: D3 to D4
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 3, TYPE] == PAWN)
        self.assertTrue(new_board[3, 3, COLOR] == WHITE)
        self.assertTrue(new_board[2, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('D7', 'E6')  # black pawn captures black pawn: D7 to E6
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_white_blocked_moving_forward_by_black(self):
        """Test white trying to move forward into a black piece."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        new_board = self.chess_game.take_turn('D7', 'D5')  # black pawn forward 2: D7 to D5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 3, TYPE] == PAWN)
        self.assertTrue(new_board[4, 3, COLOR] == BLACK)
        self.assertTrue(new_board[6, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('D4', 'D5')  # white pawn moves into black pawn: D4 to D5
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_black_blocked_moving_forward_by_white(self):
        """Test black trying to move forward into a white piece."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('D7', 'D5')  # black pawn forward 2: D7 to D5
        new_board = self.chess_game.take_turn('E2', 'E4')  # white pawn forward 2: E2 to E4
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 4, TYPE] == PAWN)
        self.assertTrue(new_board[3, 4, COLOR] == WHITE)
        self.assertTrue(new_board[1, 4, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('D5', 'D4')  # white pawn moves into black pawn: D5 to D4
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_white_blocked_moving_forward_by_white(self):
        """Test white trying to move forward into a white piece."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        self.chess_game.take_turn('E2', 'E4')  # white pawn forward 2: E2 to E4
        self.chess_game.take_turn('B7', 'B5')  # black pawn forward 2: B7 to B5
        self.chess_game.take_turn('D4', 'E5')  # white pawn captures black pawn: D4 to E5
        new_board = self.chess_game.take_turn('A7', 'A5')  # black pawn forward 2: A7 to A5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 0, TYPE] == PAWN)
        self.assertTrue(new_board[4, 0, COLOR] == BLACK)
        self.assertTrue(new_board[6, 0, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E4', 'E5')  # white pawn moves into white pawn: E4 to E5
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_black_blocked_moving_forward_by_black(self):
        """Test white trying to move forward into a white piece."""
        
        self.chess_game.take_turn('D2', 'D4')  # white pawn forward 2: D2 to D4
        self.chess_game.take_turn('E7', 'E5')  # black pawn forward 2: E7 to E5
        self.chess_game.take_turn('E2', 'E4')  # white pawn forward 2: E2 to E4
        self.chess_game.take_turn('D7', 'D5')  # black pawn forward 2: D7 to D5
        self.chess_game.take_turn('F2', 'F4')  # white pawn forward 2: F2 to F4
        self.chess_game.take_turn('E5', 'D4')  # black pawn captures white pawn: E5 to D4
        new_board = self.chess_game.take_turn('G2', 'G4')  # white pawn forward 2: G2 to G4
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 6, TYPE] == PAWN)
        self.assertTrue(new_board[3, 6, COLOR] == WHITE)
        self.assertTrue(new_board[1, 6, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('D4', 'D5')  # black pawn moves into black pawn: D4 to D5
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    # TODO: add test for promotion once apply_move() is refactored for it
        
# Load and run the test cases
pawn_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestPawns)
unittest.TextTestRunner(verbosity=1).run(pawn_test_suite)

<h2>4.3 - Rook tests</h2>
<hr>
<br>
Rooks should be fairly straightforward to test (sorry about the pun). They work the same regardless of color, so I'll just use whatever.

In [ ]:
class TestRooks(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()
        
    def test_rook_move_right_valid(self):
        """Test the rook moving to the right to an empty square."""

        self.chess_game.take_turn('B1', 'C3')  # White: Move B1 to C3 (free up the path for the rook)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5 (random valid move)
        new_board = self.chess_game.take_turn('A1', 'B1')  # White rook right 1: A1 to B1
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[0, 1, TYPE], ROOK)
        self.assertEqual(new_board[0, 1, COLOR], WHITE)
        self.assertEqual(new_board[0, 0, TYPE], EMPTY)

    def test_rook_move_left_valid(self):
        """Test the rook moving to the left to an empty square."""

        self.chess_game.take_turn('G1', 'H3')  # White: Move Knight from G1 to H3 (free up the path for the rook)
        self.chess_game.take_turn('G7', 'G5')  # Black: Move G7 to G5 (random valid move)
        new_board = self.chess_game.take_turn('H1', 'G1')  # White: Move rook from H1 to G1
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[0, 6, TYPE], ROOK)
        self.assertEqual(new_board[0, 6, COLOR], WHITE)
        self.assertEqual(new_board[0, 7, TYPE], EMPTY)

    def test_rook_move_forward_valid(self):
        """Test the rook moving forward to an empty square."""

        self.chess_game.take_turn('A2', 'A4')  # White: Move A2 to A4 (free up the rook)
        self.chess_game.take_turn('A7', 'A5')  # Black: Move A7 to A5 (random valid move)
        new_board = self.chess_game.take_turn('A1', 'A3')  # White: Move rook from A1 to A3
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 0, TYPE], ROOK)
        self.assertEqual(new_board[2, 0, COLOR], WHITE)
        self.assertEqual(new_board[0, 0, TYPE], EMPTY)

    def test_rook_move_backward_valid(self):
        """Test the rook moving backward to an empty square."""

        # Set up the board: Move the rook from D4 back to D2 (Rook move backward)
        self.chess_game.take_turn('A2', 'A4')  # White: Move A2 to A4 (make room for rook)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5 (random valid move)
        self.chess_game.take_turn('A1', 'A3')  # White: Move A1 to A3 (rook forward)
        self.chess_game.take_turn('G7', 'G5')  # Black: Move G7 to G5 (random valid move)
        new_board = self.chess_game.take_turn('A3', 'A1')  # White: Move rook from A3 back to A1
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[0, 0, TYPE], ROOK)
        self.assertEqual(new_board[0, 0, COLOR], WHITE)
        self.assertEqual(new_board[2, 0, TYPE], EMPTY)

    def test_rook_capture_right(self):
        """Test the rook capturing a piece to the right."""

        # Set up the board: Capture a black piece with the rook moving from A1 to D1
        self.chess_game.take_turn('A2', 'A4')  # White: Move A2 to A4 (free up the rook)
        self.chess_game.take_turn('D7', 'D5')  # Black: Move D7 to D5
        self.chess_game.take_turn('A1', 'A3')  # White: Move A1 to A3 (bring out the rook)
        self.chess_game.take_turn('C8', 'H3')  # Black: Move C8 to H3 (setup capture of bishop)
        new_board = self.chess_game.take_turn('A3', 'H3')  # White: Move rook from A3 to H3 and capture the Bishop
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 7, TYPE], ROOK)
        self.assertEqual(new_board[2, 7, COLOR], WHITE)
        self.assertEqual(new_board[2, 0, TYPE], EMPTY)

    def test_rook_capture_left(self):
        """Test the rook capturing a piece to the left."""

        # Set up the board: Capture a black piece with the rook moving from A1 to D1
        self.chess_game.take_turn('H2', 'H4')  # White: Move H2 to H4 (free up the rook)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        self.chess_game.take_turn('H1', 'H3')  # White: Move H1 to H3 (bring out the rook)
        self.chess_game.take_turn('F8', 'A3')  # Black: Move F8 to A3 (setup capture of bishop)
        new_board = self.chess_game.take_turn('H3', 'A3')  # White: Move rook from H3 to A3 and capture the Bishop
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 0, TYPE], ROOK)
        self.assertEqual(new_board[2, 0, COLOR], WHITE)
        self.assertEqual(new_board[2, 7, TYPE], EMPTY)

    def test_rook_capture_forward(self):
        """Test the rook capturing a piece forward."""

        # Set up the board: Capture a black pawn with the rook moving from A1 to A5
        self.chess_game.take_turn('H2', 'H4')  # White: Move H2 to H4 (free up the rook)
        self.chess_game.take_turn('D7', 'D5')  # Black: Move D7 to D5 (let the bishop out)
        self.chess_game.take_turn('D2', 'D4')  # White: Move D2 to D4 (random valid move)
        self.chess_game.take_turn('C8', 'H3')  # Black: Move C8 to H3 (setup capture of bishop)
        new_board = self.chess_game.take_turn('H1', 'H3')  # White: Move rook from H1 to H3 and capture the Bishop
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 7, TYPE], ROOK)
        self.assertEqual(new_board[2, 7, COLOR], WHITE)
        self.assertEqual(new_board[0, 7, TYPE], EMPTY)

    def test_rook_capture_backward(self):
        """Test the rook capturing a piece backward."""

        # Set up the board: Capture a black pawn with the rook moving from A1 to A5
        self.chess_game.take_turn('H2', 'H4')  # White: Move H2 to H4 (free up the rook)
        self.chess_game.take_turn('D7', 'D5')  # Black: Move D7 to D5 (allow bishop out)
        self.chess_game.take_turn('H4', 'H5')  # White: H4 to H5 (more space for rook)
        self.chess_game.take_turn('C7', 'C5')  # Black: random valid move
        self.chess_game.take_turn('H1', 'H4')  # White: H1 to H4 (ready to capture bishop)
        self.chess_game.take_turn('C8', 'H3')  # Black: Move C8 to H3 (setup capture of bishop)
        new_board = self.chess_game.take_turn('H4', 'H3')  # White: Move rook from H4 to H3 and capture the Bishop
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 7, TYPE], ROOK)
        self.assertEqual(new_board[2, 7, COLOR], WHITE)
        self.assertEqual(new_board[3, 7, TYPE], EMPTY)
        
    def test_rook_move_off_board_right(self):
        """Test the rook trying to move off the board to the right."""
        
        new_board = self.chess_game.take_turn('H1', 'H9')  # H1 to off the board
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_rook_move_off_board_left(self):
        """Test the rook trying to move off the board to the left."""

        new_board = self.chess_game.take_turn('A1', 'A0')  # A1 to off the board
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_rook_move_off_board_forward(self):
        """Test the rook trying to move off the board forward."""
        
        new_board = self.chess_game.take_turn('A2', 'A4')  # Move pawn from A2 to A4
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 0, TYPE] == PAWN)
        self.assertTrue(new_board[3, 0, COLOR] == WHITE)
        self.assertTrue(new_board[1, 0, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('A8', 'A9')  # A8 to off the board forward
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_rook_move_through_piece(self):
        """Test the rook trying to move through a blocking piece from a starting position."""
        
        new_board = self.chess_game.take_turn('A1', 'A3')  # Attempt to move the rook from A1 to A3, but A2 has a pawn blocking it
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_rook_capture_same_color(self):
        """Test the rook trying to capture a piece of the same color from a starting position."""

        new_board = self.chess_game.take_turn('A1', 'A2')  # Attempt to move the rook from A1 to A2 to capture the white pawn
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

# Load and run the test cases
rook_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestRooks)
unittest.TextTestRunner(verbosity=1).run(rook_test_suite)

<h2>4.4 - Bishop tests</h2>
<hr>
<br>
Bishops, like rooks should be fairly straightforward to test. They just move diagonally.

In [ ]:
class TestBishops(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()

    def test_bishop_move_right_diagonal_valid(self):
        """Test the bishop moving along the right diagonal to an empty square."""

        self.chess_game.take_turn('D2', 'D4')  # White: Move D2 to D4 (free up the bishop)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        new_board = self.chess_game.take_turn('C1', 'E3')  # White: Move bishop from C1 to E3
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[2, 4, TYPE], BISHOP)
        self.assertEqual(new_board[2, 4, COLOR], WHITE)
        self.assertEqual(new_board[0, 2, TYPE], EMPTY)

    def test_bishop_move_left_diagonal_valid(self):
        """Test the bishop moving along the left diagonal to an empty square."""

        self.chess_game.take_turn('E2', 'E4')  # White: Move E2 to E4 (free up the bishop)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        new_board = self.chess_game.take_turn('F1', 'C4')  # White: Move bishop from F1 to C4
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[3, 2, TYPE], BISHOP)
        self.assertEqual(new_board[3, 2, COLOR], WHITE)
        self.assertEqual(new_board[0, 5, TYPE], EMPTY)

    def test_bishop_move_blocked_by_own_piece(self):
        """Test the bishop trying to move but blocked by its own piece."""

        new_board = self.chess_game.take_turn('F1', 'D3')  # White: Try to move bishop from F1 to D3, but E2 pawn blocks the path
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_bishop_capture_right_diagonal(self):
        """Test the bishop capturing an opponent's piece along the right diagonal."""
        
        self.chess_game.take_turn('D2', 'D4')  # White: Move D2 to D4 (free up the bishop)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        self.chess_game.take_turn('C1', 'E3')  # White: Move bishop from C1 to E3
        self.chess_game.take_turn('G7', 'G5')  # Black: Move G7 to G5 (setup for capture)
        new_board = self.chess_game.take_turn('E3', 'G5')  # White: Move bishop from E3 to G5 and capture black pawn
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[4, 6, TYPE], BISHOP)
        self.assertEqual(new_board[4, 6, COLOR], WHITE)
        self.assertEqual(new_board[2, 4, TYPE], EMPTY)

    def test_bishop_capture_left_diagonal(self):
        """Test the bishop capturing an opponent's piece along the left diagonal."""

        self.chess_game.take_turn('E2', 'E4')  # White: Move E2 to E4 (free up the bishop)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        self.chess_game.take_turn('F1', 'C4')  # White: Move bishop from F1 to C4
        self.chess_game.take_turn('D7', 'D5')  # Black: Move D7 to D5 (setup for capture)
        new_board = self.chess_game.take_turn('C4', 'D5')  # White: Move bishop from C4 to D5 and capture black pawn
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[4, 3, TYPE], BISHOP)
        self.assertEqual(new_board[4, 3, COLOR], WHITE)
        self.assertEqual(new_board[3, 2, TYPE], EMPTY)

    def test_bishop_move_blocked_by_opponent_piece(self):
        """Test the bishop trying to move through an opponent's piece."""

        # Set up the board and try to move the bishop through an opponent's piece
        self.chess_game.take_turn('D2', 'D4')  # White: Move D2 to D4 (free up the bishop)
        new_board = self.chess_game.take_turn('G7', 'G5')  # Black: Move G7 to G5 (block the diagonal path)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 6, TYPE] == PAWN)
        self.assertTrue(new_board[4, 6, COLOR] == BLACK)
        self.assertTrue(new_board[6, 6, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('C1', 'H6')  # White: Try to move bishop from C1 to H6, but blocked by black's D5 pawn
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_bishop_move_and_captured(self):
        """Test a more complex scenario where the bishop moves and captures a knight before being taken by a rook"""

        # Set up the board for a complex sequence of moves and captures
        self.chess_game.take_turn('E2', 'E4')  # White: Move E2 to E4 (free up the bishop)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5
        self.chess_game.take_turn('F1', 'C4')  # White: Move bishop from F1 to C4
        self.chess_game.take_turn('F7', 'F5')  # Black: Move F7 to F5 (expose the knight)
        self.chess_game.take_turn('C4', 'G8')  # White: Move bishop from C4 to G8 and capture knight
        new_board = self.chess_game.take_turn('H8', 'G8')  # Black: Move rook from H8 to G8 and capture the bishop
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[7, 6, TYPE], ROOK)
        self.assertEqual(new_board[7, 6, COLOR], BLACK)
        self.assertEqual(new_board[7, 7, TYPE], EMPTY)
        
# Load and run the test cases
bishop_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestBishops)
unittest.TextTestRunner(verbosity=1).run(bishop_test_suite)

<h2>4.5 - Knight tests</h2>
<hr>
<br>
Knights should be fairly easy to test, they can move around in the middle and test all the moves easily enough, collisions and captures are also fairly easy to set up.

In [ ]:
class TestKnights(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()
        
    def test_knight_move_all_directions(self):
        """Test the knight moving in all directions."""
        
        self.chess_game.take_turn('B1', 'C3')  # White: Move knight from B1 to C3
        self.chess_game.take_turn('G8', 'F6')  # Black: Move knight from G8 to F6
        self.chess_game.take_turn('C3', 'B5')  # White: Move knight from C3 to B5
        self.chess_game.take_turn('F6', 'G4')  # Black: Move knight from F6 to G4
        self.chess_game.take_turn('B5', 'D4')  # White: Move knight from B5 to D4
        self.chess_game.take_turn('G4', 'E5')  # Black: Move knight from G4 to E5
        self.chess_game.take_turn('D4', 'F5')  # White: Move knight from D4 to F5
        new_board = self.chess_game.take_turn('E5', 'C4')  # Black: Move knight from E5 to C4
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[3, 2, TYPE], KNIGHT)
        self.assertEqual(new_board[3, 2, COLOR], BLACK)
        self.assertEqual(new_board[4, 4, TYPE], EMPTY)

    def test_knight_move_blocked_by_piece(self):
        """Test the knight moving to a square occupied by its own piece."""
        
        new_board = self.chess_game.take_turn('B1', 'C2')  # White: Try to move knight from B1 to C2 (blocked by own piece)
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_knight_capture_valid(self):
        """Test the knight capturing an opponent's piece."""

        self.chess_game.take_turn('G1', 'F3')  # White: Move knight from G1 to F3
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5 (setup capture)
        new_board = self.chess_game.take_turn('F3', 'E5')  # White: Move knight from F3 to E5 and capture pawn
        
        # our new board should show our piece in the new location with the old location empty
        self.assertEqual(new_board[4, 4, TYPE], KNIGHT)
        self.assertEqual(new_board[4, 4, COLOR], WHITE)
        self.assertEqual(new_board[2, 5, TYPE], EMPTY)

    def test_knight_move_invalid_straight_line(self):
        """Test the knight trying to move in a straight line (invalid move)."""
        
        new_board = self.chess_game.take_turn('B1', 'D1')  # White: Try to move knight from B1 to D1 (invalid move)
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None

# Load and run the test cases
knight_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestKnights)
unittest.TextTestRunner(verbosity=1).run(knight_test_suite)

<h2>4.6 - Queen tests</h2>
<hr>
<br>
I'm only going to be using a few tests here, but there's really nothing new to test, so I'm just lumping things together, the core logic should be captured by the rook and bishop tests.

In [ ]:
class TestQueens(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()
        
    def test_queen_movement(self):
        """Test the queen moving in all possible directions."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4 (free up the white queen)
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5 (free up the black queen)
        self.chess_game.take_turn('D1', 'G4')  # White queen: Move D1 to G4
        self.chess_game.take_turn('D8', 'G5')  # Black queen: Move D8 to G5
        self.chess_game.take_turn('G4', 'E6')  # White queen: Move G4 to E6 (check!)
        self.chess_game.take_turn('E8', 'D8')  # Black king: Move E8 to D8
        self.chess_game.take_turn('E6', 'A6')  # White: Move E6 to A6
        self.chess_game.take_turn('G5', 'E3')  # Black: Move G5 to E3 (check!)
        self.chess_game.take_turn('E1', 'D1')  # White king: Move E1 to D1
        self.chess_game.take_turn('E3', 'H3')  # Black: Move E3 to H3
        self.chess_game.take_turn('A6', 'A3')  # White: Move A6 to A3
        new_board = self.chess_game.take_turn('H3', 'H6')  # Black: Move H3 to H6

        # if we have no errors, our boards should be up to date
        self.assertEqual(new_board[5, 7, TYPE], QUEEN)
        self.assertEqual(new_board[5, 7, COLOR], BLACK)
        self.assertEqual(new_board[2, 0, TYPE], QUEEN)
        self.assertEqual(new_board[2, 0, COLOR], WHITE)

    def test_queen_move_blocked(self):
        """Test the queen moving to the left to an empty square."""
        
        new_board = self.chess_game.take_turn('D1', 'D4')  # White: Move queen from D1 to D4

        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_queen_move_like_knight(self):
        """Test the queen moving to the left to an empty square."""
        
        new_board = self.chess_game.take_turn('D1', 'A3')  # White: Move queen from D1 to A3

        self.assertIsNone(new_board)  # invalid move, new_board should be None

    def test_queen_captures_piece(self):
        """Test the queen moving forward to an empty square."""
        
        self.chess_game.take_turn('E2', 'E4')  # White: Move E2 to E4 (free up the white queen)
        self.chess_game.take_turn('E7', 'E5')  # Black: Move E7 to E5 (free up the black queen)
        self.chess_game.take_turn('D1', 'G4')  # White: Move D1 to G4
        self.chess_game.take_turn('D8', 'G5')  # Black: Move D8 to G5
        new_board = self.chess_game.take_turn('G4', 'G5')  # White: Move queen from G4 to G5 capturing the black queen

        self.assertEqual(new_board[4, 6, TYPE], QUEEN)
        self.assertEqual(new_board[4, 6, COLOR], WHITE)
        self.assertEqual(new_board[3, 6, TYPE], EMPTY)

# Load and run the test cases
queen_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestQueens)
unittest.TextTestRunner(verbosity=1).run(queen_test_suite)

<h2>4.7 - King tests</h2>
<hr>
<br>
Finally, we have our king tests. Since there's several special moves and circumstances, there will be a fair few tests here. We're going to need the basic movement, both that it can move in every direction but that it can't go any further. It needs tests for queen and king side castling. It needs tests for check, moving into check, moving out of check, checkmate and stalemate. And just to be thorough, include tests for pins, discoveries and forks, so all the catchall stuff that didn't fall into one of the previous batches of tests.

In [ ]:
class TestKings(unittest.TestCase):
    def setUp(self):
        """Set up a fresh game instance before each test."""
        
        self.chess_game = NumpyChess()
        
    def test_standard_movement(self):
        """Test the kings moving in all valid directions."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4 (let the king out)
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5 (let the king out)
        self.chess_game.take_turn('E1', 'E2')  # White king: Move E1 to E2              # move up
        self.chess_game.take_turn('E8', 'E7')  # Black king: Move E8 to E7              # move down
        self.chess_game.take_turn('E2', 'F3')  # White king: Move E2 to F3              # move up-right
        self.chess_game.take_turn('E7', 'D6')  # Black king: Move E7 to D6              # move down-left
        self.chess_game.take_turn('F3', 'G3')  # White king: Move F3 to G3              # move right
        self.chess_game.take_turn('D6', 'C6')  # Black king: Move D6 to C6              # move left
        self.chess_game.take_turn('G3', 'G4')  # White king: Move G3 to G4
        self.chess_game.take_turn('C6', 'C5')  # Black king: Move C6 to C5  
        self.chess_game.take_turn('G4', 'F5')  # White king: Move G4 to F5              # move up-left
        new_board = self.chess_game.take_turn('C5', 'D4')  # Black king: Move C5 to D4  # move down-right
        
        # if all has gone well, there have been no errors and our final locations have our kings
        self.assertEqual(new_board[3, 3, TYPE], KING)
        self.assertEqual(new_board[3, 3, COLOR], BLACK)
        self.assertEqual(new_board[4, 5, TYPE], KING)
        self.assertEqual(new_board[4, 5, COLOR], WHITE)
        
    def test_king_movement_blocked(self):
        """Test the king's move being blocked by it's own piece."""
        
        new_board = self.chess_game.take_turn('E1', 'E2')  # White king: Move E1 to E2, blocked by pawn
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_king_move_limited_to_one_square(self):
        """Test the king trying to move more than one space."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4
        new_board = self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move D7 to D5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 3, TYPE] == PAWN)
        self.assertTrue(new_board[4, 3, COLOR] == BLACK)
        self.assertTrue(new_board[6, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'E3')  # White king: Move E1 to E3
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
        
    def test_king_captures_not_from_check(self):
        """Test the king capturing a piece without being in check first."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4
        self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move D7 to D5
        self.chess_game.take_turn('D2', 'D4')  # White pawn: Move D2 to D4
        self.chess_game.take_turn('C8', 'G4')  # Black bishop: C8 to G4
        self.chess_game.take_turn('C2', 'C4')  # White pawn: Move C2 to C4
        self.chess_game.take_turn('G4', 'E2')  # Black bishop: Move G4 to E2
        new_board = self.chess_game.take_turn('E1', 'E2')  # White king: Move E1 to E2, capturing white bishop
        
        # if all has gone well, there have been no errors and our final location has our king
        self.assertEqual(new_board[1, 4, TYPE], KING)
        self.assertEqual(new_board[1, 4, COLOR], WHITE)
        self.assertEqual(new_board[0, 4, TYPE], EMPTY)
        
    def test_king_captures_to_escape_check(self):
        """Test the king capturing a piece to escape from check."""
        
        self.chess_game.take_turn('C2', 'C4')  # White pawn: Move C2 to C4
        self.chess_game.take_turn('C7', 'C5')  # Black pawn: Move C7 to C5
        self.chess_game.take_turn('H2', 'H4')  # White pawn: Move H2 to H4
        self.chess_game.take_turn('D8', 'A5')  # Black queen: D8 to A5
        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4
        self.chess_game.take_turn('A5', 'D2')  # Black queen: Move A5 to D2 (check!)
        new_board = self.chess_game.take_turn('E1', 'D2')  # White king: Move E1 to D2, capturing white queen
        
        # if all has gone well, there have been no errors and our final location has our king
        self.assertEqual(new_board[1, 3, TYPE], KING)
        self.assertEqual(new_board[1, 3, COLOR], WHITE)
        self.assertEqual(new_board[0, 4, TYPE], EMPTY)
        
    def test_king_cannot_enter_check(self):
        """Test the king trying to move into check."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4
        self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move D7 to D5
        self.chess_game.take_turn('D2', 'D4')  # White pawn: Move D2 to D4
        new_board = self.chess_game.take_turn('C8', 'G4')  # Black bishop: C8 to G4 (threatens E2)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 6, TYPE] == BISHOP)
        self.assertTrue(new_board[3, 6, COLOR] == BLACK)
        self.assertTrue(new_board[7, 2, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'E2')  # White king: Move E1 to E2, moving into check
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_cannot_capture_into_check(self):
        """Test the king trying to capture a piece, but that would put it into check."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4
        self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move D7 to D5
        self.chess_game.take_turn('E1', 'E2')  # White king: Move E1 to E2
        self.chess_game.take_turn('F7', 'F5')  # Black pawn: Move F7 to F5
        self.chess_game.take_turn('E2', 'E3')  # White king: Move E2 to E3
        new_board = self.chess_game.take_turn('D5', 'E4')  # Black pawn: Move D5 to E4, capturing white pawn
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 4, TYPE] == PAWN)
        self.assertTrue(new_board[3, 4, COLOR] == BLACK)
        self.assertTrue(new_board[4, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E2', 'E3')  # White king: Move E2 to E3, moving into check
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_cannot_capture_guarded_piece_to_escape_check(self):
        """Test the king is in check by a piece that is guarded and cannot be taken by the king."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5
        self.chess_game.take_turn('D1', 'F3')  # White queen: Move D1 to F3
        self.chess_game.take_turn('D8', 'H4')  # Black queen: Move D8 to H4
        self.chess_game.take_turn('A2', 'A4')  # White pawn: Move A2 to A4
        self.chess_game.take_turn('F8', 'C5')  # Black bishop: Move F8 to C5
        self.chess_game.take_turn('B2', 'B4')  # White pawn: Move B2 to B4
        new_board = self.chess_game.take_turn('H4', 'F2')  # Black queen: Move H4 to F2
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[1, 5, TYPE] == QUEEN)
        self.assertTrue(new_board[1, 5, COLOR] == BLACK)
        self.assertTrue(new_board[2, 7, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'F2')  # White king: Move E1 to F2, capturing queen, but it is guarded by the bishop
        
        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_cannot_move_other_piece_while_in_check(self):
        """Test the king in check prevents other moves."""
        
        self.chess_game.take_turn('E2', 'E4')  # White pawn: Move E2 to E4 (free up the white queen)
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5 (free up the black queen)
        self.chess_game.take_turn('D1', 'G4')  # White queen: Move D1 to G4
        self.chess_game.take_turn('D8', 'G5')  # Black queen: Move D8 to G5
        new_board = self.chess_game.take_turn('G4', 'E6')  # White queen: Move G4 to E6 (check!)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[5, 4, TYPE] == QUEEN)
        self.assertTrue(new_board[5, 4, COLOR] == WHITE)
        self.assertTrue(new_board[3, 6, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('G5', 'E3')  # Black: Move G5 to E3

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle(self):
        """Test the king castling on the king's side."""
        
        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4 (let the bishop out)
        self.chess_game.take_turn('G7', 'G5')  # Black pawn: Move G7 to G5 (let the bishop out)
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3 (move the bishop out of the way)
        self.chess_game.take_turn('F8', 'H6')  # Black bishop: Move F8 to H6 (move the bishop out of the way)
        self.chess_game.take_turn('G1', 'F3')  # White knight: Move G1 to F3 (move the knight out of the way)
        self.chess_game.take_turn('G8', 'F6')  # Black knight: Move G8 to F6 (move the knight out of the way)
        self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle)
        new_board = self.chess_game.take_turn('E8', 'G8')  # Black king: Move E8 to G8 (castle)

        # if all has gone well, there have been no errors and our final locations have our rooks after castling
        self.assertEqual(new_board[0, 5, TYPE], ROOK)
        self.assertEqual(new_board[0, 5, COLOR], WHITE)
        self.assertEqual(new_board[0, 4, TYPE], EMPTY)
        self.assertEqual(new_board[7, 5, TYPE], ROOK)
        self.assertEqual(new_board[7, 5, COLOR], BLACK)
        self.assertEqual(new_board[7, 4, TYPE], EMPTY)
    
    def test_king_side_castle_blocked_by_king_in_check(self):
        """Test the king-side castling impossible if in check."""

        self.chess_game.take_turn('G2', 'G3')  # White pawn: Move G2 to G3 (let the bishop out)
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5 (let the bishop out)
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3 (move the bishop out of the way)
        self.chess_game.take_turn('F7', 'F5')  # Black pawn: Move F7 to F5 (random valid move)
        self.chess_game.take_turn('G1', 'F3')  # White knight: Move G1 to F3 (move the knight out of the way)
        self.chess_game.take_turn('G7', 'G5')  # Black pawn: Move G7 to G5 (random valid move)
        self.chess_game.take_turn('D2', 'D4')  # White pawn: Move D2 to D4 (expose the king)
        new_board = self.chess_game.take_turn('F8', 'B4')  # Black bishop: Move F8 to B4 (check!)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 1, TYPE] == BISHOP)
        self.assertTrue(new_board[3, 1, COLOR] == BLACK)
        self.assertTrue(new_board[7, 5, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle)

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_blocked_by_king_moving_across_check(self):
        """Test the king-side castling impossible if moving across check."""
        
        self.chess_game.take_turn('G2', 'G3')  # White pawn: Move G2 to G3 (let the bishop out)
        self.chess_game.take_turn('G7', 'G5')  # Black pawn: Move G7 to G5 (let the bishop out)
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3 (move the bishop out of the way)
        self.chess_game.take_turn('F8', 'H6')  # Black bishop: Move F8 to H6 (move the bishop out of the way)
        self.chess_game.take_turn('G1', 'F3')  # White knight: Move G1 to F3 (move the knight out of the way)
        self.chess_game.take_turn('D7', 'D6')  # Black pawn: Move D7 to D6 (move the pawn out of the bishop's way)
        self.chess_game.take_turn('D2', 'D4')  # White pawn: Move D2 to D4 (random valid move)
        new_board = self.chess_game.take_turn('C8', 'H3')  # Black bishop: Move D7 to D6 (move the pawn out of the bishop's way)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[2, 7, TYPE] == BISHOP)
        self.assertTrue(new_board[2, 7, COLOR] == BLACK)
        self.assertTrue(new_board[7, 2, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle)

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_blocked_by_king_moving_into_check(self):
        """Test the king-side castling impossible if moving into check."""

        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4 (let the bishop out)
        self.chess_game.take_turn('E7', 'E6')  # Black pawn: Move E7 to E6 (let the bishop out)
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3 (move the bishop out of the way)
        self.chess_game.take_turn('F8', 'D6')  # Black bishop: Move F8 to H6 (move the bishop out of the way)
        self.chess_game.take_turn('G1', 'F3')  # White knight: Move G1 to F3 (move the knight out of the way)
        new_board = self.chess_game.take_turn('D6', 'H2')  # Black bishop: Move D4 to H2, capturing white pawn and threatening G2
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[1, 7, TYPE] == BISHOP)
        self.assertTrue(new_board[1, 7, COLOR] == BLACK)
        self.assertTrue(new_board[3, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle)

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_blocked_by_own_piece_in_way(self):
        """Test the king-side castling when one of the same colored pieces in the way."""

        self.assertIsNotNone(self.chess_game.take_turn('G1', 'F3'))  # White knight: Move G1 to F3 (move the knight out of the way)
        new_board = self.chess_game.take_turn('E7', 'E6')  # Black pawn: Move E7 to E6 (random valid move)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[5, 4, TYPE] == PAWN)
        self.assertTrue(new_board[5, 4, COLOR] == BLACK)
        self.assertTrue(new_board[6, 4, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle), blocked by white bishop

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_cannot_capture(self):
        """Test the king-side castling attempt to capture a piece."""

        self.chess_game.take_turn('F2', 'F4')  # White pawn: Move F2 to F4
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5
        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4
        self.chess_game.take_turn('F8', 'C5')  # Black bishop: Move E7 to E5
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move G2 to G4
        new_board = self.chess_game.take_turn('C5', 'G1')  # Black bishop: Move E7 to E5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[0, 6, TYPE] == BISHOP)
        self.assertTrue(new_board[0, 6, COLOR] == BLACK)
        self.assertTrue(new_board[4, 2, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle), blocked by black bishop

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_blocked_by_king_moved_before(self):
        """Test the king-side castle failure if king has already moved."""

        self.chess_game.take_turn('F2', 'F4')  # White pawn: Move F2 to F4
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5
        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4
        self.chess_game.take_turn('F8', 'C5')  # Black bishop: Move F8 to C5
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3
        self.chess_game.take_turn('C5', 'G1')  # Black bishop: Move C5 to G1
        self.chess_game.take_turn('E1', 'F1')  # White king: Move E1 to F1
        self.chess_game.take_turn('G1', 'C5')  # Black bishop: Move G1 to C5
        self.chess_game.take_turn('F1', 'E1')  # White king: Move F1 back to E1
        new_board = self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move D7 to D5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 3, TYPE] == PAWN)
        self.assertTrue(new_board[4, 3, COLOR] == BLACK)
        self.assertTrue(new_board[6, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle), king has already moved

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_king_side_castle_blocked_by_rook_moved_before(self):
        """Test the king-side castle failure if rook has already moved."""

        self.chess_game.take_turn('F2', 'F4')  # White pawn: Move F2 to F4
        self.chess_game.take_turn('E7', 'E5')  # Black pawn: Move E7 to E5
        self.chess_game.take_turn('G2', 'G4')  # White pawn: Move G2 to G4
        self.chess_game.take_turn('F8', 'C5')  # Black bishop: Move F8 to C5
        self.chess_game.take_turn('F1', 'H3')  # White bishop: Move F1 to H3
        self.chess_game.take_turn('C5', 'G1')  # Black bishop: Move C5 to G1 capturing white knight
        self.chess_game.take_turn('H1', 'G1')  # White rook: Move H1 to G1 capturing black bishop
        self.chess_game.take_turn('D7', 'D5')  # Black pawn: Move G1 to C5
        self.chess_game.take_turn('G1', 'H1')  # White rook: Move G1 back to H1
        new_board = self.chess_game.take_turn('C7', 'C5')  # Black pawn: Move D7 to D5
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 2, TYPE] == PAWN)
        self.assertTrue(new_board[4, 2, COLOR] == BLACK)
        self.assertTrue(new_board[6, 2, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'G1')  # White king: Move E1 to G1 (castle), king has already moved

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_queen_side_castle(self):
        """Test the king castling on the queen's side."""
        
        self.chess_game.take_turn('D2', 'D4')  # White pawn
        self.chess_game.take_turn('D7', 'D5')  # Black pawn
        self.chess_game.take_turn('C1', 'E3')  # White bishop
        self.chess_game.take_turn('C8', 'E6')  # Black bishop
        self.chess_game.take_turn('D1', 'D3')  # White queen
        self.chess_game.take_turn('D8', 'D6')  # Black queen
        self.chess_game.take_turn('B1', 'A3')  # White knight
        self.chess_game.take_turn('B8', 'A6')  # Black knight
        self.chess_game.take_turn('E1', 'C1')  # White king: castle queen side
        new_board = self.chess_game.take_turn('E8', 'C8')  # Black king: castle queen side
        
        # if all has gone well, there have been no errors and our final locations have our rooks after castling
        self.assertEqual(new_board[0, 3, TYPE], ROOK)
        self.assertEqual(new_board[0, 3, COLOR], WHITE)
        self.assertEqual(new_board[0, 0, TYPE], EMPTY)
        self.assertEqual(new_board[7, 3, TYPE], ROOK)
        self.assertEqual(new_board[7, 3, COLOR], BLACK)
        self.assertEqual(new_board[7, 0, TYPE], EMPTY)
    
    def test_queen_side_castle_blocked_by_king_in_check(self):
        """Test the queen-side castling impossible if in check."""
        
        self.chess_game.take_turn('D2', 'D4')  # White pawn
        self.chess_game.take_turn('D7', 'D5')  # Black pawn
        self.chess_game.take_turn('C1', 'E3')  # White bishop
        self.chess_game.take_turn('C8', 'E6')  # Black bishop
        self.chess_game.take_turn('D1', 'D3')  # White queen
        self.chess_game.take_turn('D8', 'D6')  # Black queen
        self.chess_game.take_turn('B1', 'A3')  # White knight
        new_board = self.chess_game.take_turn('D6', 'B4')  # Black queen (check!)
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 1, TYPE] == QUEEN)
        self.assertTrue(new_board[3, 1, COLOR] == BLACK)
        self.assertTrue(new_board[5, 3, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'C1')  # White king: Move E1 to C1 (castle), but king is in check

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_queen_side_castle_blocked_by_king_moving_across_check(self):
        """Test the queen-side castling impossible if moving across check."""
        
        self.chess_game.take_turn('D2', 'D4')  # White pawn
        self.chess_game.take_turn('D7', 'D5')  # Black pawn
        self.chess_game.take_turn('C1', 'E3')  # White bishop
        self.chess_game.take_turn('D8', 'D7')  # Black queen
        self.chess_game.take_turn('D1', 'D3')  # White queen
        self.chess_game.take_turn('D7', 'A4')  # Black queen
        self.chess_game.take_turn('B1', 'A3')  # White knight
        self.chess_game.take_turn('C7', 'C5')  # Black pawn
        self.chess_game.take_turn('C2', 'C4')  # White pawn
        new_board = self.chess_game.take_turn('B7', 'B5')  # Black pawn
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[4, 1, TYPE] == PAWN)
        self.assertTrue(new_board[4, 1, COLOR] == BLACK)
        self.assertTrue(new_board[6, 1, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'C1')  # White king: Move E1 to C1 (castle), but king is moving across check

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_queen_side_castle_blocked_by_king_moving_into_check(self):
        """Test the queen-side castling impossible if moving into check."""
        
        self.chess_game.take_turn('D2', 'D4')  # White pawn
        self.chess_game.take_turn('D7', 'D5')  # Black pawn
        self.chess_game.take_turn('C1', 'E3')  # White bishop
        self.chess_game.take_turn('D8', 'D7')  # Black queen
        self.chess_game.take_turn('D1', 'D3')  # White queen
        self.chess_game.take_turn('D7', 'A4')  # Black queen
        self.chess_game.take_turn('B1', 'A3')  # White knight
        self.chess_game.take_turn('C7', 'C5')  # Black pawn
        self.chess_game.take_turn('C2', 'C4')  # White pawn
        new_board = self.chess_game.take_turn('A4', 'C4')  # Black queen: capture white pawn
        
        # if everything went well, the last move should have gone through
        self.assertTrue(new_board[3, 2, TYPE] == QUEEN)
        self.assertTrue(new_board[3, 2, COLOR] == BLACK)
        self.assertTrue(new_board[3, 0, TYPE] == EMPTY)
        
        new_board = self.chess_game.take_turn('E1', 'C1')  # White king: Move E1 to C1 (castle), but king is moving into check

        self.assertIsNone(new_board)  # invalid move, new_board should be None
    
    def test_queen_side_castle_blocked_by_own_piece_in_way(self):
        """Test the queen-side castling when one of the same colored pieces in the way."""
        pass
    
    def test_queen_side_castle_cannot_capture(self):
        """Test the queen-side castling attempt to capture a piece."""
        pass
    
    def test_queen_side_castle_blocked_by_king_moved_before(self):
        """Test the queen-side castle failure if king has already moved."""
        pass
    
    def test_queen_side_castle_blocked_by_rook_moved_before(self):
        """Test the queen-side castle failure if rook has already moved."""
        pass
    
    def test_checkmate(self):
        """Test the king cannot escape, game over."""
        pass
    
    def test_stalemate(self):
        """Test the kings are all that's left, no winner possible."""
        pass
    
    def test_pin_restricts_piece_movement(self):
        """Test the piece being pinned cannot move and put the king in check."""
        pass
    
    def test_discovery_triggers_check(self):
        """Test the king being put into check by a different piece via discovery still works."""
        pass
    
    def test_fork_blocks_escape(self):
        """Test that forks prevent saving the other piece."""
        pass
    
    def test_check_ended_by_other_piece_capturing_threatening_piece(self):
        """Test that another piece can take the piece putting the king in check."""
        pass

# Load and run the test cases
king_test_suite = unittest.TestLoader().loadTestsFromTestCase(TestKings)
unittest.TextTestRunner(verbosity=1).run(king_test_suite)

<h1>Part 5 - Getting the Data</h1><br>

Here is where we will be sifting through our huge set of played chess games to find the games we actually want to train a model on. As for what makes a game desirable, first we need to be able to label it accurately. White wins, black wins or stalemate are the only ones we're interested in, ongoing games don't give us any useful information. Next we should probably decide on a skill level to train on. At first glance, the best ranked players may be the obvious choice, but I'm less interested in them. I want the other games too, the ones where people make stupid moves and we see what happens when people make stupid moves. We probably also want the better players, but we may want to balance the ratios better. Training against games where one player is noticably stronger than the other also has some interesting potential.<br>
<br>
Once we have the games converted into numpy arrays, it will be time for feature engineering. My general idea was to train some models on lots of curated played games and build a classifier model that will predict the likelihood of each player winning based on a given board and then just picking the move whose board has the highest chance of winning. No need to teach the AI anything about the rules of chess, we'll just be running a prediction based on a numpy array representing a board and some whatever features I manage to pull out and picking the one with the best odds. I may have to train a white model and a black model and mash them together into the final product, but that's not the end of the world.<br>
<br>
I'll come back to this section when I get a bit further on the refactoring.

<h1>Part 6 - The Refactoring</h1><br>
So now that I have a much better idea of what it needs to do and what it needs to be able to load, time to refactor it. 
So this needs to be rewritten now that I think I know what it needs to do beyond just be a chessboard. In particular, it needs to be able to take a move in algebraic notation, not just the rank and file locations, which is what we'll use for any kind of human interactive game format.<br>

<h2>6.1 - Constants and Enums</h2><br>
The main change here is that I'm breaking things into Enums instead of just having a bunch of global variables, mostly just for readability, but it also lets me pass information back with a MoveResult.<br>
<br>
For MoveResult, we have various error conditions and turn states. For Color, we have black, white and empty. For PieceType we have each of the pieces plus empty. 

In [ ]:
class MoveResult(Enum):
    ERROR_CHECK = 0
    ERROR_NO_PIECE = 1
    ERROR_WRONG_COLOR = 2
    ERROR_CANNOT_REACH = 3
    WHITE_TURN = 4
    BLACK_TURN = 5
    BLACK_TURN_CHECK = 6
    WHITE_TURN_CHECK = 7
    BLACK_WINS = 8
    WHITE_WINS = 9

class Color(Enum):
    EMPTY = 0
    WHITE = 1
    BLACK = 2
    
class PieceType(Enum):
    EMPTY = 0
    PAWN = 1
    ROOK = 2
    KNIGHT = 3
    BISHOP = 4
    QUEEN = 5
    KING = 6

# move-tuple indexes
MOVE_START_ROW, MOVE_START_COL, MOVE_END_ROW, MOVE_END_COL = 0, 1, 2, 3
# piece indexes
TYPE, COLOR = 0, 1
# position indexes
LOCATION_ROW, LOCATION_COLUMN = 0, 1
# move directions for pieces
BISHOP_MOVE_DIRECTIONS = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
ROOK_MOVE_DIRECTIONS = [(1, 0), (-1, 0), (0, 1), (0, -1)]
ROYAL_MOVE_DIRECTIONS = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
EMPTY_PIECE = np.array([Color.EMPTY, PieceType.EMPTY])

<h2>6.2 - The Pieces</h2><br>
Next up, we have our chess pieces. Aside from readability, these are mostly just to convert the board into a numpy array at the end, the logic is all in the game and board. The argument could be made for including things like get_possible_moves(self) or something like it on the ChessPiece class, but to see where a piece can move relies on other pieces so really doesn't belong here. Other than that, the pieces have a PieceType and a Color. You could potentially have a has_moved() method or member variable on the piece as well, which I think would make sense as well, but I'm going to keep that logic in the game itself so the pieces and board can potentially be reused outside of the standard game.

In [ ]:
class ChessPiece:
    def __init__(self, color: Color, piece_type: PieceType):
        self.color = color
        self.piece_type = piece_type
        
    def to_numpy(self) -> ndarray:
        return np.array([self.color.value, self.piece_type.value])
    
class Pawn(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.PAWN)
        
class Rook(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.ROOK)
        
class Knight(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.KNIGHT)
        
class Bishop(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.BISHOP)
        
class Queen(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.QUEEN)
        
class King(ChessPiece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.KING)

<h2>6.3 - The Board</h2><br>
Next up, the board will get its own class. This will only keep track of the current pieces in each square and give information about the board. For example, the board could tell you if a player is in check, or which square a knight could move to, but not whose turn it is or if the rook has moved before, it has no memory of such things. Things like en passant and castling will be handled as if they are valid, their validity won't be checked here, that will have to be handled by the game itself that tracks such things. This will let us use the board for any other test scenarios we may want to add, some of which may be invalid setups (checking for errors in game histories for example), or just custom scenarios. Basically this will be as dumb as possible while still giving us what information it can, I'm going to try to offload the bulk of the heavy lifting to the ChessGame class below.<br>
<br>
__init__(self): Currently our constructor just fires up a fresh game all set up, but that's something that could be passed in later for a custom board.<br>
to_numpy(self): We will need to be able to convert any given board to its numpy array representation.<br>
piece_at(self, location): We need to be able to ask what piece is at a given square.<br>
get_possible_moves(self, location): We will need to be able to get a list of potential moves for a piece at a given location. This doesn't include any logic for check, just if the piece could make it there, the check logic is done in the game.<br>
is_in_check(self, color): returns true if the player is in check, false otherwise.
preview_move(self, origin, destination): this will return a MoveResult and a new board for that move, but the board will not be updated. This lets us know what the board would look like if the move were to take place and if it would return an error.<br>
make_move(self, origin, destination): this returns the MoveResult and a new board just like the preview method does, but the move is finalized and the game board is updated.

In [ ]:
class ChessBoard:
    def __init__(self):
        self.board: ndarray = np.empty((8, 8), dtype=object)
            
        # place pawns
        for col in range(8):
            self.board[1, col] = Pawn(Color.WHITE)
            self.board[6, col] = Pawn(Color.BLACK)
        # rooks
        self.board[0, [0, 7]] = Rook(Color.WHITE)
        self.board[7, [0, 7]] = Rook(Color.BLACK)
        # knights
        self.board[0, [1, 6]] = Knight(Color.WHITE)
        self.board[7, [1, 6]] = Knight(Color.BLACK)
        # bishops
        self.board[0, [2, 5]] = Bishop(Color.WHITE)
        self.board[7, [2, 5]] = Bishop(Color.BLACK)
        # queens
        self.board[0, 3] = Queen(Color.WHITE)
        self.board[7, 3] = Queen(Color.BLACK)
        # kings
        self.board[0, 4] = King(Color.WHITE)
        self.board[7, 4] = King(Color.BLACK)
        
    def to_numpy(self) -> ndarray:
        result: ndarray = np.zeros((8, 8, 2), dtype=int)
        for row in range(8):
            for col in range(8):
                piece: Optional[ChessPiece] = self.board[row, col]
                if piece:
                    result[row, col] = piece.to_numpy()
                else:
                    result[row, col] = np.array([Color.EMPTY.value, PieceType.EMPTY.value])
        
        return result
    
    # get the piece at the location or None
    def piece_at(self, location: Tuple[int, int]) -> Optional[ChessPiece]:
        piece: Optional[ChessPiece] = self.board[location[0], location[1]]
        return piece
    
    def get_possible_moves(self, piece: ChessPiece) -> List[Tuple[int, int]]:
        possible_moves: List[Tuple[int, int]] = []

        # Helper function to add a move if valid
        def add_move_if_valid(row: int, col: int):
            if 0 <= row < 8 and 0 <= col < 8:
                dest_piece: Optional[ChessPiece] = self.piece_at((row, col))
                if dest_piece is None or dest_piece.color != piece.color:
                    possible_moves.append((row, col))

        if piece.piece_type == PieceType.PAWN:
            direction: int = 1 if piece.color == Color.WHITE else -1
            piece_in_front: Optional[ChessPiece] = self.piece_at((piece.row + direction, piece.col))
            # Normal move
            if piece_in_front is None:
                possible_moves.append((piece.row + direction, piece.col))
            # Double move from starting position
            if (piece.color == Color.WHITE and piece.row == 1) or (piece.color == Color.BLACK and piece.row == 6):
                if piece_in_front is None and self.piece_at((piece.row + direction * 2, piece.col)) is None:
                    possible_moves.append((piece.row + direction * 2, piece.col))
            # Capture moves
            for dc in [-1, 1]:
                new_col: int = piece.col + dc
                new_row: int = piece.row + direction
                if 0 <= new_col < 8:
                    dest_piece: Optional[ChessPiece] = self.piece_at((new_row, new_col))
                    if dest_piece is not None and dest_piece.color != piece.color:
                        possible_moves.append((new_row, new_col))
                    # only check for en passant if the square is empty
                    elif dest_piece is None:
                        adjacent_piece: Optional[ChessPiece] = self.piece_at((piece.row, new_col))
                        behind_dest_piece: Optional[ChessPiece] = self.piece_at((new_row + direction, new_col))
                        if adjacent_piece and adjacent_piece.piece_type == PieceType.PAWN and not behind_dest_piece:
                            possible_moves.append((new_row, new_col))

        elif piece.piece_type == PieceType.ROOK:
            for dr, dc in ROOK_MOVE_DIRECTIONS:
                row: int = piece.row
                col: int = piece.col
                while True:
                    row += dr
                    col += dc
                    if 0 <= row < 8 and 0 <= col < 8:
                        dest_piece: Optional[ChessPiece] = self.piece_at((row, col))
                        if dest_piece is None:
                            possible_moves.append((row, col))
                        elif dest_piece.color != piece.color:
                            possible_moves.append((row, col))
                            break
                        else:
                            break
                    else:
                        break

        elif piece.piece_type == PieceType.KNIGHT:
            for dr, dc in [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]:
                add_move_if_valid(piece.row + dr, piece.col + dc)

        elif piece.piece_type == PieceType.BISHOP:
            for dr, dc in BISHOP_MOVE_DIRECTIONS:
                row: int = piece.row
                col: int = piece.col
                while True:
                    row += dr
                    col += dc
                    if 0 <= row < 8 and 0 <= col < 8:
                        dest_piece: Optional[ChessPiece] = self.piece_at((row, col))
                        if dest_piece is None:
                            possible_moves.append((row, col))
                        elif dest_piece.color != piece.color:
                            possible_moves.append((row, col))
                            break
                        else:
                            break
                    else:
                        break

        elif piece.piece_type == PieceType.QUEEN:
            for dr, dc in ROYAL_MOVE_DIRECTIONS:
                row: int = piece.row
                col: int = piece.col
                while True:
                    row += dr
                    col += dc
                    if 0 <= row < 8 and 0 <= col < 8:
                        dest_piece: Optional[ChessPiece] = self.piece_at((row, col))
                        if dest_piece is None:
                            possible_moves.append((row, col))
                        elif dest_piece.color != piece.color:
                            possible_moves.append((row, col))
                            break
                        else:
                            break
                    else:
                        break

        elif piece.piece_type == PieceType.KING:
            for dr, dc in ROYAL_MOVE_DIRECTIONS:
                add_move_if_valid(piece.row + dr, piece.col + dc)
            start_row: int = 0 if piece.color == Color.WHITE else 7
            # only possible to castle if in original position
            if piece.row == start_row and piece.col == 4:
                # check king side conditions
                rook: Optional[ChessPiece] = self.piece_at((start_row, 7))
                knight_square: Optional[ChessPiece] = self.piece_at((start_row, 6))
                bishop_square: Optional[ChessPiece] = self.piece_at((start_row, 5))
                if rook and rook.color == piece.color and not knight_square and not bishop_square:
                    possible_moves.append((start_row, 6))
                
                # check queen side conditions
                rook = self.piece_at((start_row, 0))
                knight_square = self.piece_at((start_row, 1))
                bishop_square = self.piece_at((start_row, 2))
                queen_square: Optional[ChessPiece] = self.piece_at((start_row, 3))
                if rook and rook.color == piece.color and not knight_square and not bishop_square and not queen_square:
                    possible_moves.append((start_row, 2))

        return possible_moves
    
    def is_in_check(self, color: Color) -> bool:
        pass
    
    def preview_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> Tuple[MoveResult, Optional[ndarray]]:
        pass
    
    def make_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> Tuple[MoveResult, Optional[ndarray]]:
        pass

<h2>6.4 - The Game</h2><br>
Here we have the core logic that figures out valid moves, turn tracking, the UI, and anything else that got tacked on.<br>
<br>
So let's go through take_turn() a little closer:<br>
<br>
1. we make sure we have a valid starting piece.<br>
2. we see if that piece can move there. Our board has no idea of whether en passant or castling is possible in this case since it has no history. It will include castling and en passant if potentially possible and leave it to us here to perform final validation.<br>
3. If this move is a castle, we check to make sure it's valid by comparing it to the variables we track in step 8.<br>
4. If this move is an en passant, we compare our destination to the en_passant_square we track in step 7.<br>
5. Now that it's been validated as a legal move, we tell our board to go ahead and make the move.<br>
6. If that didn't return an error because it moved into check, then we now have our updated board which we add to our history numpy array. This array is what we'll be using to chop up into data for training and testing sets.<br>
7. If this is a pawn moving ahead 2, then it is vulnerable next turn to en passant and set the skipped over square to our en_passant_square, otherwise en passant is not possible next turn so set it to None.<br>
8. And finally, moving a king or rook will invalidate future castlings of the appropriate type(s).

In [ ]:
class ChessGame:
    def __init__(self):
        self.board: ChessBoard = ChessBoard()
        self.history: ndarray = np.zeros((1, 8, 8, 2), dtype=int)
        self.history[0] = self.board.to_numpy()
        self.player_turn: Color = Color.WHITE
            
        # if a pawn moves 2, they are vulnerable to en passant, but only after just moving
        self.en_passant_square: Optional[Tuple[int, int]] = None
        
        # if the pieces move, these change, this is just faster than calculating it each time
        self.black_king_castle: bool = True
        self.black_queen_castle: bool = True
        self.white_king_castle: bool = True
        self.white_queen_castle: bool = True
    
    def take_turn(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> MoveResult:
        origin_piece: Optional[ChessPiece] = self.board.piece_at(origin)
        destination_piece: Optional[ChessPiece] = self.board.piece_at(destination)
        
        # make sure we have a valid piece
        if not origin_piece:
            print('No piece at that position to move', origin, destination)
            return MoveResult.ERROR_NO_PIECE
        if not origin_piece.color == self.player_turn:
            print('Not your piece', origin, destination)
            return MoveResult.ERROR_WRONG_COLOR
        
        # the list of all possible moves, including en passant and castling
        destination_list: List[Tuple[int, int]] = self.board.get_possible_moves(origin_piece)
        
        # that piece cannot reach the destination in any scenario
        if destination not in destination_list:
            print('That piece cannot move there', origin, destination)
            return MoveResult.ERROR_CANNOT_REACH
        
        # if it is a king, check if it's a castle
        if origin_piece.piece_type == PieceType.KING and is_castle(origin, destination):
            # piece where the rook will land
            middlePiece: Optional[ChessPiece] = self.board.piece_at((origin[0], (origin[1] + destination[1]) / 2))
            # piece beside the rook if queen side castling
            internal_piece: Optional[ChessPiece] = None if destination[1] == 6 else self.board.piece_at((origin[0], 1))
                
            # castling is only possible if the destination and intervening pieces are empty
            if middle_piece or destination_piece or internal_piece:
                print('Illegal castle, piece in the way')
                return MoveResult.ERROR_CANNOT_REACH
            
            # check if that particular castle is possible
            if origin == (0, 4) and dest == (0, 2):
                if not self.white_king_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (0, 4) and dest == (0, 6):
                if not self.white_queen_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (7, 4) and dest == (7, 2):
                if not self.black_king_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (7, 4) and dest == (7, 6):
                if not self.black_queen_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
        
        # if it is an en passant, validate it
        if origin_piece.piece_type == PieceType.PAWN and origin_piece.can_capture(destination) and self.board.piece_at(destination) is None:
            if self.en_passant_square != destination:
                print('Illegal move, no en passant possible and no piece to take', origin, destination)
                return MoveResult.ERROR_CANNOT_REACH
        
        # otherwise make the move
        move_result, new_board = self.board.make_move(origin, destination)
                
        # if it's moving into check, report error
        if move_result == MoveResult.ERROR_CHECK:
            print('Invalid move, that would put you in check', origin, destination)
            return MoveResult.ERROR_CHECK
        
        # update the move history
        new_board = new_board.reshape(1, 8, 8, 2)
        self.history = np.concatenate((self.history, new_board), axis=0)
            
        # track en passant, only possible after a pawn first moves 2 spaces and is the space that was jumped over, otherwise clear it
        if origin_piece.piece_type == PieceType.PAWN:
            if origin_piece.color == Color.WHITE and origin[0] == 1 and destination[0] == 3:
                self.en_passant_square = (2, origin[1])
            elif origin_piece.color == Color.BLACK and origin[0] == 6 and destination[0] == 4:
                self.en_passant_square = (5, origin[1])
            else:
                self.en_passant_square = None
        else:
            self.en_passant_square = None
            
        # rook and king moves make castling unavailable
        if origin == (0, 4):
            self.white_king_castle = False
            self.white_queen_castle = False
        elif origin == (0, 0):
            self.white_queen_castle = False
        elif origin == (0, 7):
            self.white_king_castle = False
        elif origin == (7, 4):
            self.black_king_castle = False
            self.black_queen_castle = False
        elif origin == (7, 0):
            self.black_queen_castle = False
        elif origin == (7, 7):
            self.black_king_castle = False
                
        # return the move result
        return move_result
    
    def take_turn_uci(self, uci_move: str) -> MoveResult:
        pass
    
    def display_board(self) -> None:
        pass

Now to start testing and debugging it:

In [ ]:
newGame = ChessGame()